# What happens:
Files are created, send email with excel script

## Import 

In [1]:
import os
import sys
import pandas as pd
import numpy as np
from pandas import DataFrame
import sqlalchemy
from sqlalchemy import create_engine
import getpass
import datetime
import win32com.client as win32
import calendar
from openpyxl import Workbook, load_workbook
from IPython.lib.display import Audio
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.cell import Cell
from openpyxl.styles import Color, Font, PatternFill
from zipfile import ZipFile

### Select the folder where you wanna do the work
### Also ask for the Reporting Date

In [2]:
directory = os.path.abspath(os.getcwd()) + "/"

rd = str(input("Please enter the RD (YYYYMMDD): "))

### Make connection with Oracle DB

In [3]:
#Setting up RAY connection:
connection_RAY = create_engine(str("oracle://:@OCRPN"))
connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'),{partition})".format(
                 reporting_date = rd, partition = 2))

## Function

In [4]:
def save_files(file_names, ws_nr, sqls):
    # Create file
    for index, file_name in enumerate(file_names):
        df = pd.DataFrame()
        print("Trying " + str(file_names[index]))
        try:
            if(ws_nr[index] != 0):
                connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'), {partition})".format(
                        reporting_date = rd, partition = ws_nr[index]))
            else:
                connection_RAY.execute("call pack_context.context_disable()")
                
            df = pd.read_sql_query( sqls[index], connection_RAY)
        except:
            print( str(file_names[index]) + " is not created")
            
        for col in df.select_dtypes(include=['datetime64[ns]']):
            df[col] = pd.to_datetime(df[col]).dt.date
                
        df.to_excel("Export_" + file_name + "_" + rd[:6] + ".xlsx", index = False)
        print('DataFrame is written to Excel File successfully.')
    print('Files are ready')


In [5]:
def save_file(file_names, ex_folder, ws_nr, sqls):
   
    dfs = []
    # Select SQLs
    for index, file_name in enumerate(file_names):
        print("Trying " + str(file_name))
        try:
            connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'), {partition})".format(
                        reporting_date = rd, partition = ws_nr[index]))
            df_temp= pd.read_sql_query( sqls[index], connection_RAY)
            df_temp = df_temp.replace(";", "-")
            dfs.append(df_temp)
        except:
            dfs.append(pd.DataFrame())
            print( str(ws_nr[index]) + " is not created")
            
    for index, file_name in enumerate(file_names):
        dfs[index].to_csv(ex_folder + "Export_" + file_name + "_" + rd[:6] + ".csv", sep = ';', index = False)
        
    print('DataFrame is written to Excel File successfully.')


### DQIs

In [6]:
ex_folder = "Export/"
file_names = ["4.2.2.1.45", "4.2.2.3.38", "4.2.2.3.39", "4.2.2.3.40", "4.2.2.3.46", "4.2.2.3.47", "4.2.2.3.48", "4.2.2.1.19A", "RI-GROUP.08.10.02.0017", "RWA.014", "RWA.007", "RWA.015", "RWA.016" ]
ws_nr = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 194, 2, 2]
sqls = ["""-- 4.2.2.1.45
with     CCN_GROUP_EXPOSURE as (
          -- Total group exposure per cocunut (CCN_GROUP_EXPOSURE)
   SELECT   nvl(nvl(CDR.COUNTERPARTY_NEW_CODE, CDR.COUNTERPARTY_CODE),'UNKNOWN') AS COUNTERPARTY,
             round(SUM(nvl(CDR.EAD_PRE_CCF,0)),4) AS CCN_GROUP_EXPOSURE,
             round(SUM(nvl(CDR.RWA,0)),4) as CCN_GROUP_RWA,
             nvl(CDR.METHOD_NEW, CDR.METHOD_ORIGINAL) AS METHOD, 
             nvl(CDR.SUB_METHOD_NEW,CDR.METHOD_ORIGINAL) AS SUB_METHOD,
             nvl(CDR.DEDUCTIONS,0) as TIER1_DEDUCTION,
             nvl(CDR.DEDUCTIONS,0) as TIER2_DEDUCTION,
             CDR.BS_TYPE
   FROM     V_RZB_CDR CDR
            WHERE CDR.RECORD_SOURCE='T_CDR'
            AND PARTITION_KEY = '"""+rd+"""10002'
   GROUP BY PARTITION_KEY, nvl(nvl(CDR.COUNTERPARTY_NEW_CODE, CDR.COUNTERPARTY_CODE),'UNKNOWN'), nvl(METHOD_NEW, METHOD_ORIGINAL), nvl(CDR.SUB_METHOD_NEW,CDR.METHOD_ORIGINAL), nvl(CDR.DEDUCTIONS,0), CDR.BS_TYPE
         ),
      CCN_EXPOSURE_NWU as (
   -- Total exposure per cocunut and NWU (CCN_EXPOSURE_NWU)
   SELECT   RANK() OVER (PARTITION BY nvl(nvl(CDR.COUNTERPARTY_NEW_CODE, CDR.COUNTERPARTY_CODE),'UNKNOWN') ORDER BY SUM(CDR.EAD_PRE_CCF) DESC) as Rank,
            nvl(nvl(CDR.COUNTERPARTY_NEW_CODE, CDR.COUNTERPARTY_CODE),'UNKNOWN') AS COUNTERPARTY,
            round(SUM(nvl(CDR.EAD_PRE_CCF,0)),4) AS CCN_EXPOSURE_NWU,
            substr(CDR.BOOK_CODE,1,instr(CDR.BOOK_CODE, '_')-1) as BOOKING_UNIT
   FROM     V_RZB_CDR CDR
            WHERE CDR.RECORD_SOURCE='T_CDR'
            AND PARTITION_KEY = '"""+rd+"""0002'
   GROUP BY PARTITION_KEY, nvl(nvl(CDR.COUNTERPARTY_NEW_CODE, CDR.COUNTERPARTY_CODE),'UNKNOWN'), 
            substr(CDR.BOOK_CODE,1,instr(CDR.BOOK_CODE, '_')-1)
      ),
      CCN_EXPOSURE_GPC_NWU as (
         -- Total exposure per cocunut, NWU and GPC (CCN_EXP_GPC_NWU)
   SELECT   RANK() OVER (PARTITION BY nvl(nvl(CDR.COUNTERPARTY_NEW_CODE, CDR.COUNTERPARTY_CODE),'UNKNOWN') ORDER BY SUM(CDR.EAD_PRE_CCF) DESC) as Rank,
            nvl(nvl(CDR.COUNTERPARTY_NEW_CODE, CDR.COUNTERPARTY_CODE),'UNKNOWN') AS COUNTERPARTY,
            round(SUM(nvl(CDR.EAD_PRE_CCF,0)),4) AS CCN_EXP_GPC_NWU,
            CDR.CONTRACT_TYPE,
            substr(CDR.BOOK_CODE,1,instr(CDR.BOOK_CODE, '_')-1) as BOOKING_UNIT
   FROM     V_RZB_CDR CDR
            WHERE CDR.RECORD_SOURCE='T_CDR'
            AND PARTITION_KEY = '"""+rd+"""0002'
   GROUP BY PARTITION_KEY, nvl(nvl(CDR.COUNTERPARTY_NEW_CODE, CDR.COUNTERPARTY_CODE),'UNKNOWN'), 
            CDR.CONTRACT_TYPE, substr(CDR.BOOK_CODE,1,instr(CDR.BOOK_CODE, '_')-1)
      ),
      CCN_RATING as (
   -- Cocunut with rating
   SELECT   Distinct ENTITY.ENTITY_CODE AS COCUNUT,
            ENTITY.ENTITY_CODE_2 AS MAIN_CUSTOMER_ID,
            ENTITY.ENTITY_DESC as CUSTOMER_NAME,
            ENTITY.ET_CODE_USER as ASSET_CLASS_ID,
            ENTITY_TYPE.ENTITY_TYPE_DESC as ASSET_CLASS_DESC,
            replace(CUSTOMER_SOURCE.UNIT, 'GR_','') AS CUSTOMER_UNIT,
            ICR.AGENCY_CODE,
            substr(ICR.LONG_TERM_RATING,1,INSTR(ICR.LONG_TERM_RATING, '_')-1) AS RAT_MOD,
             substr(ICR.LONG_TERM_RATING,INSTR(ICR.LONG_TERM_RATING, '_')+1, 3) AS RATING,
             decode(SUBSTR(ICR.LONG_TERM_RATING,INSTR(ICR.LONG_TERM_RATING, '_')+1, 3), 'MIS', 'Y', NULL, 'Y', 'N') AS MIS_RATING,
            ICR.RATING_DATE,
            ENTITY.DEFAULT_FLAG
   FROM     ENTITY ENTITY 
        INNER JOIN ENTITY_TYPE ENTITY_TYPE 
            ON ENTITY.ET_CODE_USER = ENTITY_TYPE.ENTITY_TYPE
            AND ENTITY_TYPE.PARTITION_KEY = '"""+rd+"""0000'
        LEFT JOIN (SELECT substr(DESCRIPTION, instr(DESCRIPTION, '+')+1, length(DESCRIPTION)) as UNIT, CODE FROM IMPORT_SOURCE) CUSTOMER_SOURCE 
            ON ENTITY.IMPORT_SOURCE = CUSTOMER_SOURCE.CODE 
        LEFT JOIN ISSUER_CREDIT_RATINGS ICR 
            ON ENTITY.ENTITY_CODE = ICR.ENTITY_CODE 
            AND ICR.AGENCY_CODE = 'INTERNAL'
            AND ICR.PARTITION_KEY = '"""+rd+"""0000'
        WHERE ENTITY.PARTITION_KEY = '"""+rd+"""0000'
      ), FRM_ACTIVE_COCUNUT AS (
SELECT   Distinct ENTITY.ENTITY_CODE AS COCUNUT,
         ENTITY.ENTITY_CODE_2 AS MAIN_CUSTOMER_ID,
         ENTITY.ENTITY_DESC as CUSTOMER_NAME,
         ENTITY.ET_CODE_USER as ASSET_CLASS_ID,
         ENTITY_TYPE.ENTITY_TYPE_DESC as ASSET_CLASS_DESC,
         replace(CUSTOMER_SOURCE.UNIT, 'GR_','') AS CUSTOMER_UNIT,
         ENTITY.DEFAULT_FLAG
FROM     ENTITY ENTITY 
        INNER JOIN ENTITY_TYPE ENTITY_TYPE 
            ON ENTITY.ET_CODE_USER = ENTITY_TYPE.ENTITY_TYPE
            AND ENTITY_TYPE.PARTITION_KEY = '"""+rd+"""0000'
        LEFT JOIN (SELECT substr(DESCRIPTION, instr(DESCRIPTION, '+') + 1, length(DESCRIPTION)) as UNIT, CODE FROM IMPORT_SOURCE) CUSTOMER_SOURCE 
            ON ENTITY.IMPORT_SOURCE = CUSTOMER_SOURCE.CODE
        WHERE ENTITY.STATUS = 'A'
        AND ENTITY.PARTITION_KEY = '"""+rd+"""0000'
), FRM_EXPCOCRAT AS (
SELECT   distinct CCN_GROUP_EXPOSURE.COUNTERPARTY AS COCUNUT,
         CCN_RATING.MAIN_CUSTOMER_ID,
         CCN_RATING.CUSTOMER_NAME,
         CCN_RATING.ASSET_CLASS_ID,
         CCN_RATING.ASSET_CLASS_DESC,
         CCN_RATING.CUSTOMER_UNIT,
         CCN_RATING.DEFAULT_FLAG,
         CCN_RATING.RATING,
         CCN_RATING.RAT_MOD,
         CCN_RATING.RATING_DATE,
         CCN_RATING.MIS_RATING,
         CCN_RATING.AGENCY_CODE,
         CCN_GROUP_EXPOSURE.CCN_GROUP_EXPOSURE,
         nvl(CCN_GRP_EXP_ASS.CCN_GROUP_EXPOSURE, 0) AS CCN_GRP_EXP_ASS,
         nvl(CCN_GRP_EXP_OFB.CCN_GROUP_EXPOSURE, 0) AS CCN_GRP_EXP_OFB,
         CCN_GROUP_EXPOSURE.CCN_GROUP_RWA,
         nvl(CCN_GRP_EXP_ASS.CCN_GROUP_RWA, 0) AS CCN_GRP_RWA_ASS,
         nvl(CCN_GRP_EXP_OFB.CCN_GROUP_RWA, 0) AS CCN_GRP_RWA_OFB,
         nvl(CCN_GROUP_EXPOSURE.TIER1_DEDUCTION,0) as TIER1_DEDUCTION,
         nvl(CCN_GROUP_EXPOSURE.TIER2_DEDUCTION,0) as TIER2_DEDUCTION,
         CCN_MAX_EXPOSURE_NWU.CCN_MAX_EXPOSURE_NWU,
         CCN_MAX_EXPOSURE_NWU.BOOKING_UNIT AS UNIT_MAX_EXPOSURE,
         CCN_EXP_MAX_GPC_NWU.CCN_MAX_EXP_GPC_NWU,
         CCN_EXP_MAX_GPC_NWU.MAX_GPC as MAX_GPC_CODE,
         CCN_EXP_MAX_GPC_NWU.BOOKING_UNIT AS UNIT_MAX_GPC,
         nvl(CCN_EXPOSURE_HO.EXPOSURE_IN_HO,0) AS EXPOSURE_IN_HO,
         nvl(CCN_EXPOSURE_APPROACH.CCN_GRP_EXP_STD_TOTAL,0) AS CCN_GRP_EXP_STD_TOTAL,
         nvl(CCN_EXPOSURE_APPROACH.CCN_GRP_EXP_STD_DED,0) AS CCN_GRP_EXP_STD_DED,
         nvl(CCN_EXPOSURE_APPROACH.CCN_GRP_EXP_STD_SFA,0) AS CCN_GRP_EXP_STD_SFA,
         nvl(CCN_EXPOSURE_APPROACH.CCN_GRP_EXP_STD_STD,0) AS CCN_GRP_EXP_STD_STD,
         nvl(CCN_EXPOSURE_APPROACH.CCN_GRP_EXP_STD_NO_SUBMETHOD,0) AS CCN_GRP_EXP_STD_NO_SUBMETHOD,
         nvl(CCN_EXPOSURE_APPROACH.CCN_GRP_EXP_FOU_TOTAL,0) AS CCN_GRP_EXP_FOU_TOTAL,
         nvl(CCN_EXPOSURE_APPROACH.CCN_GRP_EXP_FOU_C,0) AS CCN_GRP_EXP_FOU_C,
         nvl(CCN_EXPOSURE_APPROACH.CCN_GRP_EXP_FOU_DED,0) AS CCN_GRP_EXP_FOU_DED,
         nvl(CCN_EXPOSURE_APPROACH.CCN_GRP_EXP_FOU_PDA,0) AS CCN_GRP_EXP_FOU_PDA,
         nvl(CCN_EXPOSURE_APPROACH.CCN_GRP_EXP_FOU_RBA,0) AS CCN_GRP_EXP_FOU_RBA,
         nvl(CCN_EXPOSURE_APPROACH.CCN_GRP_EXP_FOU_SRW,0) AS CCN_GRP_EXP_FOU_SRW,
         nvl(CCN_EXPOSURE_APPROACH.CCN_GRP_EXP_FOU_STD,0) AS CCN_GRP_EXP_FOU_STD,
         nvl(CCN_EXPOSURE_APPROACH.CCN_GRP_EXP_FOU_NO_SUBMETHOD,0) AS CCN_GRP_EXP_FOU_NO_SUBMETHOD, 
         nvl(CCN_EXPOSURE_APPROACH.CCN_GRP_RWA_STD_TOTAL,0) AS CCN_GRP_RWA_STD_TOTAL,
         nvl(CCN_EXPOSURE_APPROACH.CCN_GRP_RWA_STD_DED,0) AS CCN_GRP_RWA_STD_DED,
         nvl(CCN_EXPOSURE_APPROACH.CCN_GRP_RWA_STD_SFA,0) AS CCN_GRP_RWA_STD_SFA,
         nvl(CCN_EXPOSURE_APPROACH.CCN_GRP_RWA_STD_STD,0) AS CCN_GRP_RWA_STD_STD,
         nvl(CCN_EXPOSURE_APPROACH.CCN_GRP_RWA_STD_NO_SUBMETHOD,0) AS CCN_GRP_RWA_STD_NO_SUBMETHOD,
         nvl(CCN_EXPOSURE_APPROACH.CCN_GRP_RWA_FOU_TOTAL,0) AS CCN_GRP_RWA_FOU_TOTAL,
         nvl(CCN_EXPOSURE_APPROACH.CCN_GRP_RWA_FOU_C,0) AS CCN_GRP_RWA_FOU_C,
         nvl(CCN_EXPOSURE_APPROACH.CCN_GRP_RWA_FOU_DED,0) AS CCN_GRP_RWA_FOU_DED,
         nvl(CCN_EXPOSURE_APPROACH.CCN_GRP_RWA_FOU_PDA,0) AS CCN_GRP_RWA_FOU_PDA,
         nvl(CCN_EXPOSURE_APPROACH.CCN_GRP_RWA_FOU_RBA,0) AS CCN_GRP_RWA_FOU_RBA,
         nvl(CCN_EXPOSURE_APPROACH.CCN_GRP_RWA_FOU_SRW,0) AS CCN_GRP_RWA_FOU_SRW,
         nvl(CCN_EXPOSURE_APPROACH.CCN_GRP_RWA_FOU_STD,0) AS CCN_GRP_RWA_FOU_STD,
         nvl(CCN_EXPOSURE_APPROACH.CCN_GRP_RWA_FOU_NO_SUBMETHOD,0) AS CCN_GRP_RWA_FOU_NO_SUBMETHOD
FROM     (  SELECT   COUNTERPARTY,
            SUM(CCN_GROUP_EXPOSURE) AS CCN_GROUP_EXPOSURE,
             SUM(CCN_GROUP_RWA) as CCN_GROUP_RWA,
             SUM(TIER1_DEDUCTION) as TIER1_DEDUCTION,
             SUM(TIER2_DEDUCTION) as TIER2_DEDUCTION
            FROM     CCN_GROUP_EXPOSURE
            GROUP BY COUNTERPARTY 
          ) CCN_GROUP_EXPOSURE
LEFT JOIN CCN_RATING ON CCN_GROUP_EXPOSURE.COUNTERPARTY = CCN_RATING.COCUNUT
LEFT JOIN ( -- MAXIMUM EXPOSURE PER COCUNUT AND NWU (CCN_MAX_EXPOSURE_NWU)
   SELECT   COUNTERPARTY, CCN_EXPOSURE_NWU AS CCN_MAX_EXPOSURE_NWU, BOOKING_UNIT
   FROM     CCN_EXPOSURE_NWU
   WHERE    RANK = 1
   AND      CCN_EXPOSURE_NWU > 0
    ) CCN_MAX_EXPOSURE_NWU ON CCN_GROUP_EXPOSURE.COUNTERPARTY = CCN_MAX_EXPOSURE_NWU.COUNTERPARTY
LEFT JOIN ( -- MAXIMUM EXPOSURE PER COCUNUT, NWU AND GPC (CCN_MAX_EXP_GPC_NWU)
   SELECT   COUNTERPARTY, CCN_EXP_GPC_NWU AS CCN_MAX_EXP_GPC_NWU, min(CONTRACT_TYPE) AS MAX_GPC, BOOKING_UNIT
   FROM     CCN_EXPOSURE_GPC_NWU
   WHERE    RANK = 1 
   AND      CCN_EXP_GPC_NWU > 0
   GROUP BY COUNTERPARTY, CCN_EXP_GPC_NWU, BOOKING_UNIT
    ) CCN_EXP_MAX_GPC_NWU ON CCN_GROUP_EXPOSURE.COUNTERPARTY = CCN_EXP_MAX_GPC_NWU.COUNTERPARTY
LEFT JOIN ( -- Exposure in HO
   SELECT   COUNTERPARTY, CCN_EXPOSURE_NWU AS EXPOSURE_IN_HO
   FROM     CCN_EXPOSURE_NWU
   WHERE    BOOKING_UNIT = 'HO' 
    ) CCN_EXPOSURE_HO ON CCN_GROUP_EXPOSURE.COUNTERPARTY = CCN_EXPOSURE_HO.COUNTERPARTY
LEFT JOIN ( -- Exposure by Approach
      SELECT    CCN_GROUP_EXPOSURE.COUNTERPARTY, 
                STD.CCN_EXPOSURE AS CCN_GRP_EXP_STD_TOTAL,
                STD_DED.CCN_EXPOSURE AS CCN_GRP_EXP_STD_DED,
                STD_SFA.CCN_EXPOSURE AS CCN_GRP_EXP_STD_SFA,
                STD_STD.CCN_EXPOSURE AS CCN_GRP_EXP_STD_STD,
                STD_NULL.CCN_EXPOSURE AS CCN_GRP_EXP_STD_NO_SUBMETHOD,
                FOU.CCN_EXPOSURE AS CCN_GRP_EXP_FOU_TOTAL,
                FOU_C.CCN_EXPOSURE AS CCN_GRP_EXP_FOU_C,
                FOU_DED.CCN_EXPOSURE AS CCN_GRP_EXP_FOU_DED,
                FOU_PDA.CCN_EXPOSURE AS CCN_GRP_EXP_FOU_PDA,
                FOU_RBA.CCN_EXPOSURE AS CCN_GRP_EXP_FOU_RBA,
                FOU_SRW.CCN_EXPOSURE AS CCN_GRP_EXP_FOU_SRW,
                FOU_STD.CCN_EXPOSURE AS CCN_GRP_EXP_FOU_STD,
                FOU_NULL.CCN_EXPOSURE AS CCN_GRP_EXP_FOU_NO_SUBMETHOD, 
                STD.CCN_RWA AS CCN_GRP_RWA_STD_TOTAL,
                STD_DED.CCN_RWA AS CCN_GRP_RWA_STD_DED,
                STD_SFA.CCN_RWA AS CCN_GRP_RWA_STD_SFA,
                STD_STD.CCN_RWA AS CCN_GRP_RWA_STD_STD,
                STD_NULL.CCN_RWA AS CCN_GRP_RWA_STD_NO_SUBMETHOD,
                FOU.CCN_RWA AS CCN_GRP_RWA_FOU_TOTAL,
                FOU_C.CCN_RWA AS CCN_GRP_RWA_FOU_C,
                FOU_DED.CCN_RWA AS CCN_GRP_RWA_FOU_DED,
                FOU_PDA.CCN_RWA AS CCN_GRP_RWA_FOU_PDA,
                FOU_RBA.CCN_RWA AS CCN_GRP_RWA_FOU_RBA,
                FOU_SRW.CCN_RWA AS CCN_GRP_RWA_FOU_SRW,
                FOU_STD.CCN_RWA AS CCN_GRP_RWA_FOU_STD,
                FOU_NULL.CCN_RWA AS CCN_GRP_RWA_FOU_NO_SUBMETHOD
           FROM      CCN_GROUP_EXPOSURE
            LEFT JOIN ( SELECT   COUNTERPARTY, SUM(NVL(CCN_GROUP_EXPOSURE, 0)) AS CCN_EXPOSURE, SUM(NVL(CCN_GROUP_RWA, 0)) AS CCN_RWA
                       FROM     CCN_GROUP_EXPOSURE
                       WHERE    METHOD = 'STD'
                       GROUP BY COUNTERPARTY
                  ) STD ON CCN_GROUP_EXPOSURE.COUNTERPARTY = STD.COUNTERPARTY
            LEFT JOIN ( SELECT   COUNTERPARTY, SUM(NVL(CCN_GROUP_EXPOSURE, 0)) AS CCN_EXPOSURE, SUM(NVL(CCN_GROUP_RWA, 0)) AS CCN_RWA
                        FROM     CCN_GROUP_EXPOSURE
                       WHERE    METHOD = 'STD'
                       AND      SUB_METHOD = 'DED'
                       GROUP BY COUNTERPARTY
               ) STD_DED ON CCN_GROUP_EXPOSURE.COUNTERPARTY = STD_DED.COUNTERPARTY
            LEFT JOIN ( SELECT   COUNTERPARTY, SUM(NVL(CCN_GROUP_EXPOSURE, 0)) AS CCN_EXPOSURE, SUM(NVL(CCN_GROUP_RWA, 0)) AS CCN_RWA
                       FROM     CCN_GROUP_EXPOSURE
                       WHERE    METHOD = 'STD'
                       AND      SUB_METHOD = 'SFA'
                       GROUP BY COUNTERPARTY
               ) STD_SFA ON CCN_GROUP_EXPOSURE.COUNTERPARTY = STD_SFA.COUNTERPARTY
            LEFT JOIN ( SELECT   COUNTERPARTY, SUM(NVL(CCN_GROUP_EXPOSURE, 0)) AS CCN_EXPOSURE, SUM(NVL(CCN_GROUP_RWA, 0)) AS CCN_RWA
                       FROM     CCN_GROUP_EXPOSURE
                       WHERE    METHOD = 'STD'
                       AND      SUB_METHOD = 'STD'
                       GROUP BY COUNTERPARTY
               ) STD_STD ON CCN_GROUP_EXPOSURE.COUNTERPARTY = STD_STD.COUNTERPARTY
            LEFT JOIN ( SELECT   COUNTERPARTY, SUM(NVL(CCN_GROUP_EXPOSURE, 0)) AS CCN_EXPOSURE, SUM(NVL(CCN_GROUP_RWA, 0)) AS CCN_RWA
                       FROM     CCN_GROUP_EXPOSURE
                       WHERE    METHOD = 'STD'
                       AND      SUB_METHOD IS NULL
                       GROUP BY COUNTERPARTY
               ) STD_NULL ON CCN_GROUP_EXPOSURE.COUNTERPARTY = STD_NULL.COUNTERPARTY
            LEFT JOIN ( SELECT   COUNTERPARTY, SUM(NVL(CCN_GROUP_EXPOSURE, 0)) AS CCN_EXPOSURE, SUM(NVL(CCN_GROUP_RWA, 0)) AS CCN_RWA
                       FROM     CCN_GROUP_EXPOSURE
                       WHERE    METHOD = 'FOU'
                       GROUP BY COUNTERPARTY
               ) FOU ON CCN_GROUP_EXPOSURE.COUNTERPARTY = FOU.COUNTERPARTY
            LEFT JOIN ( SELECT   COUNTERPARTY, SUM(NVL(CCN_GROUP_EXPOSURE, 0)) AS CCN_EXPOSURE, SUM(NVL(CCN_GROUP_RWA, 0)) AS CCN_RWA
                       FROM     CCN_GROUP_EXPOSURE
                       WHERE    METHOD = 'FOU'
                       AND      SUB_METHOD = 'C'
                       GROUP BY COUNTERPARTY
               ) FOU_C ON CCN_GROUP_EXPOSURE.COUNTERPARTY = FOU_C.COUNTERPARTY
            LEFT JOIN ( SELECT   COUNTERPARTY, SUM(NVL(CCN_GROUP_EXPOSURE, 0)) AS CCN_EXPOSURE, SUM(NVL(CCN_GROUP_RWA, 0)) AS CCN_RWA
                       FROM     CCN_GROUP_EXPOSURE
                       WHERE    METHOD = 'FOU'
                       AND      SUB_METHOD = 'DED'
                       GROUP BY COUNTERPARTY
               ) FOU_DED ON CCN_GROUP_EXPOSURE.COUNTERPARTY = FOU_DED.COUNTERPARTY
            LEFT JOIN ( SELECT   COUNTERPARTY, SUM(NVL(CCN_GROUP_EXPOSURE, 0)) AS CCN_EXPOSURE, SUM(NVL(CCN_GROUP_RWA, 0)) AS CCN_RWA
                       FROM     CCN_GROUP_EXPOSURE
                       WHERE    METHOD = 'FOU'
                       AND      SUB_METHOD = 'PDA'
                       GROUP BY COUNTERPARTY
               ) FOU_PDA ON CCN_GROUP_EXPOSURE.COUNTERPARTY = FOU_PDA.COUNTERPARTY
            LEFT JOIN ( SELECT   COUNTERPARTY, SUM(NVL(CCN_GROUP_EXPOSURE, 0)) AS CCN_EXPOSURE, SUM(NVL(CCN_GROUP_RWA, 0)) AS CCN_RWA
                       FROM     CCN_GROUP_EXPOSURE
                       WHERE    METHOD = 'FOU'
                       AND      SUB_METHOD = 'RBA'
                       GROUP BY COUNTERPARTY
               ) FOU_RBA ON CCN_GROUP_EXPOSURE.COUNTERPARTY = FOU_RBA.COUNTERPARTY
            LEFT JOIN ( SELECT   COUNTERPARTY, SUM(NVL(CCN_GROUP_EXPOSURE, 0)) AS CCN_EXPOSURE, SUM(NVL(CCN_GROUP_RWA, 0)) AS CCN_RWA
                       FROM     CCN_GROUP_EXPOSURE
                       WHERE    METHOD = 'FOU'
                       AND      SUB_METHOD = 'SRW'
                       GROUP BY COUNTERPARTY
               ) FOU_SRW ON CCN_GROUP_EXPOSURE.COUNTERPARTY = FOU_SRW.COUNTERPARTY
            LEFT JOIN ( SELECT   COUNTERPARTY, SUM(NVL(CCN_GROUP_EXPOSURE, 0)) AS CCN_EXPOSURE, SUM(NVL(CCN_GROUP_RWA, 0)) AS CCN_RWA
                       FROM     CCN_GROUP_EXPOSURE
                       WHERE    METHOD = 'FOU'
                       AND      SUB_METHOD = 'STD'
                       GROUP BY COUNTERPARTY
               ) FOU_STD ON CCN_GROUP_EXPOSURE.COUNTERPARTY = FOU_STD.COUNTERPARTY
            LEFT JOIN ( SELECT   COUNTERPARTY, SUM(NVL(CCN_GROUP_EXPOSURE, 0)) AS CCN_EXPOSURE, SUM(NVL(CCN_GROUP_RWA, 0)) AS CCN_RWA
                       FROM     CCN_GROUP_EXPOSURE
                       WHERE    METHOD = 'FOU'
                       AND      SUB_METHOD is NULL
                       GROUP BY COUNTERPARTY
               ) FOU_NULL ON CCN_GROUP_EXPOSURE.COUNTERPARTY = FOU_NULL.COUNTERPARTY
    ) CCN_EXPOSURE_APPROACH ON CCN_GROUP_EXPOSURE.COUNTERPARTY = CCN_EXPOSURE_APPROACH.COUNTERPARTY
LEFT JOIN ( SELECT   COUNTERPARTY,
               SUM(CCN_GROUP_EXPOSURE) AS CCN_GROUP_EXPOSURE,
               SUM(CCN_GROUP_RWA) as CCN_GROUP_RWA
            FROM     CCN_GROUP_EXPOSURE
            WHERE    BS_TYPE = 'ASS'
            GROUP BY COUNTERPARTY
          ) CCN_GRP_EXP_ASS ON CCN_GROUP_EXPOSURE.COUNTERPARTY = CCN_GRP_EXP_ASS.COUNTERPARTY
LEFT JOIN ( SELECT   COUNTERPARTY,
               SUM(CCN_GROUP_EXPOSURE) AS CCN_GROUP_EXPOSURE,
               SUM(CCN_GROUP_RWA) as CCN_GROUP_RWA
            FROM     CCN_GROUP_EXPOSURE
            WHERE    BS_TYPE = 'OFB'
            GROUP BY COUNTERPARTY
          ) CCN_GRP_EXP_OFB ON CCN_GROUP_EXPOSURE.COUNTERPARTY = CCN_GRP_EXP_OFB.COUNTERPARTY)
SELECT  UNIT, COCUNUT, MAIN_CUSTOMER_ID, CUSTOMER_NAME, ASSET_CLASS_ID, ASSET_CLASS_DESC, CCN_GROUP_EXPOSURE, CCN_GROUP_RWA, UNIT_MAX_EXPOSURE, CCN_MAX_EXPOSURE_NWU
FROM     (
       SELECT FRM_ACTIVE_COCUNUT.CUSTOMER_UNIT as UNIT, FRM_ACTIVE_COCUNUT.COCUNUT, FRM_ACTIVE_COCUNUT.MAIN_CUSTOMER_ID, FRM_ACTIVE_COCUNUT.CUSTOMER_NAME, FRM_ACTIVE_COCUNUT.ASSET_CLASS_ID, FRM_ACTIVE_COCUNUT.ASSET_CLASS_DESC, FRM_EXPCOCRAT.CCN_GROUP_EXPOSURE, FRM_EXPCOCRAT.CCN_GROUP_RWA, FRM_EXPCOCRAT.UNIT_MAX_EXPOSURE, FRM_EXPCOCRAT.CCN_MAX_EXPOSURE_NWU 
    FROM     FRM_ACTIVE_COCUNUT
    LEFT JOIN FRM_EXPCOCRAT on FRM_ACTIVE_COCUNUT.COCUNUT = FRM_EXPCOCRAT.COCUNUT
    WHERE    FRM_ACTIVE_COCUNUT.ASSET_CLASS_ID IN ('5','05','13','14','16','17','29','41','43','44','63','64','65') -- defect 1404

  )
WHERE UNIT IS NOT NULL
ORDER BY UNIT""", 
"""-- 4.2.2.3.38
select distinct s.UNIT,
s.DEFAULT_ID,
s.CROSS_DEFAULT_COCUNUT,
t.entity_type, 
t.entity_type_desc,
s.DEFAULT_STATUS_DETAIL,
s.CROSS_DEFAULT_START_DATE,
s.TDC,
s.TDC2,
s.DQI_CHECK
from 
(with sub1 as (
select  cross_default.default_id AS default_id,
 cross_default.cocunut AS cross_default_cocunut,
 cross_default.default_status_detail AS default_status_detail,
cross_default.start_date as cross_default_start_date,
 cross_default.booking_company AS UNIT,
 trigger_default.cocunut AS  tdc,
 trigger_default_2.cocunut AS tdc2,
 cross_default.partition_key AS cross_default_partition_key 
from 
 (SELECT partition_key,cocunut, default_start_confirmed_on, start_date, default_classification_1,booking_company, default_id, default_status_detail
     FROM rzb_default 
     where default_status ='A' 
  and default_status_detail in ('P','C','E')  
  and default_classification_1 in ('D12')
 -- and partition_key = to_char(to_date('#DQI_PROC_DATETIME#','DDmonYYYY'),'YYYYMMDD')||'0000'  
  ) cross_default 
 left  join 
 (SELECT cocunut, default_start_confirmed_on, nvl(default_end_date,to_date('01.01.2199','DD.MM.YYYY')) end_date ,default_classification_1
     FROM rzb_default 
     where default_status ='A' 
  and default_status_detail in ('P','C','E')  
  and default_classification_1 in ('D1','D2','D3','D4','D5','D6','D7','D8','D9','D10','D11')
   --and partition_key = to_char(to_date('#DQI_PROC_DATETIME#','DDmonYYYY'),'YYYYMMDD')||'0000' 
  ) trigger_default
 on cross_default.cocunut = trigger_default.cocunut 
 and cross_default.default_start_confirmed_on between (trigger_default.default_start_confirmed_on-7) and (trigger_default.end_date + 7)
 left  join 
 (SELECT cocunut, start_date, nvl(default_end_date,to_date('01.01.2199','DD.MM.YYYY')) end_date, default_classification_1
     FROM rzb_default 
     where default_status ='A' 
  and default_status_detail in ('P','C','E')  
  and default_classification_1 in ('D1','D2','D3','D4','D5','D6','D7','D8','D9','D10','D11')
   --and partition_key = to_char(to_date('#DQI_PROC_DATETIME#','DDmonYYYY'),'YYYYMMDD')||'0000' 
  ) trigger_default_2 
 on cross_default.cocunut = trigger_default_2.cocunut 
 and cross_default.start_date between (trigger_default_2.start_date-7) 
 and (trigger_default_2.end_date + 7) 
)
select 
UNIT,
DEFAULT_ID,
CROSS_DEFAULT_COCUNUT,
DEFAULT_STATUS_DETAIL,
CROSS_DEFAULT_START_DATE,
TDC,
TDC2 ,
case when  sub1.tdc is null and sub1.tdc2 is null then 'ERROR' else 'OK' end as DQI_CHECK
from sub1
--left join entity e on s.cross_default_cocunut = e.entity_code
--left join entity_type t on e.et_code_user = t.entity_type 
where Trunc(cross_default_start_date) > to_date('2016-01-01','yyyy-mm-dd')
--and sub1.cross_default_partition_key = to_char(to_date('#DQI_PROC_DATETIME#','DDmonYYYY'),'YYYYMMDD')||'0000'  
order by 1) s
left join entity e on s.cross_default_cocunut = e.entity_code
left join entity_type  t on t.entity_type= e.et_code_user
""",
"""-- 4.2.2.3.39
with sub1 as (      
select  trigger_default.default_id trigger_default_id,      
 trigger_default.cocunut  trigger_default_cocunut,      
 trigger_default.entity_type  trigger_default_entity_type, 
 trigger_default.entity_type_desc  trigger_default_entity_type_desc, 
 trigger_default.default_start_confirmed_on  trigger_default_start_conf_on,      
 trigger_default.default_classification_1  trigger_default_class_1,      
trigger_default.booking_company trigger_booking_company,      
 trigger_default.rank trigger_default_rank,      
 cross_default.partition_key cross_default_partition_key,      
 cross_default.default_id Cross_default_id,      
 cross_default.cocunut  cross_default_cocunut,    
 cross_default.entity_type  cross_default_entity_type, 
 cross_default.entity_type_desc  cross_default_entity_type_desc,  
 cross_default.default_start_confirmed_on  cross_default_start_conf_on,      
 cross_default.default_classification_1  cross_default_class_1,      
 cross_default.booking_company cross_booking_company,      
 cross_default.rank cross_default_rank,      
 abs(trigger_default.default_start_confirmed_on - cross_default.default_start_confirmed_on) number_days      
from       
 (select partition_key, default_id, cocunut, entity_type, entity_type_desc, default_start_confirmed_on, default_classification_1, rank, booking_company        
  from (SELECT d.partition_key, d.default_id, d.cocunut, t.entity_type as entity_type, t.entity_type_desc as entity_type_desc, d.default_start_confirmed_on, d.default_classification_1, d.booking_company,ROW_NUMBER()      
      OVER (PARTITION BY d.cocunut ORDER BY to_number(d.default_id) asc ) AS rank      
      FROM rzb_default d
left join entity e on d.cocunut = e.entity_code
left join entity_type  t on t.entity_type= e.et_code_user      
   where d.default_status ='A'       
   and d.default_status_detail in ('P','C','E')
   --and partition_key = to_char(to_date('#DQI_PROC_DATETIME#','DDmonYYYY'),'YYYYMMDD')||'0000'   
   and Trunc(d.start_date) > to_date('2016-01-01','yyyy-mm-dd')    
  ) where default_classification_1 in ('D1','D2','D3','D4','D5','D6','D7','D8','D9','D10','D11')      
  ) trigger_default       
 inner join       
 (select partition_key, default_id, cocunut, entity_type, entity_type_desc, default_start_confirmed_on, default_classification_1,rank,booking_company       
  from (SELECT d.partition_key, d.default_id, d.cocunut, t.entity_type as entity_type, t.entity_type_desc as entity_type_desc, d.default_start_confirmed_on, d.default_classification_1, d.booking_company, ROW_NUMBER()      
      OVER (PARTITION BY  d.cocunut ORDER BY to_number(d.default_id) asc ) AS rank      
      FROM rzb_default  d
 left join entity e on d.cocunut = e.entity_code
 left join entity_type  t on t.entity_type= e.et_code_user     
   where d.default_status ='A'
   --and partition_key = to_char(to_date('#DQI_PROC_DATETIME#','DDmonYYYY'),'YYYYMMDD')||'0000'       
   and d.default_status_detail in ('P','C','E')      
   and Trunc(d.start_date) > to_date('2016-01-01','yyyy-mm-dd')
  ) where default_classification_1 in ('D12')      
  ) cross_default       
  on trigger_default.cocunut = cross_default.cocunut      
-- where abs(trigger_default.rank - cross_default.rank) < 2       
 -- and trigger_default.rank < cross_default.rank      
-- and abs(trigger_default.default_start_confirmed_on - cross_default.default_start_confirmed_on) > 4      
-- and cross_default.default_start_confirmed_on > to_date(to_Char(substr(cross_default.partition_key, 0, 6)) || '01', 'YYYY-MM-DD')      
)      
----------------      
-- DETAIL SQL --      
----------------      
select 
trigger_booking_company AS UNIT,
trigger_default_id,      
trigger_default_cocunut,  
trigger_default_entity_type, 
trigger_default_entity_type_desc,
--to_char (trigger_default_start_conf_on, 'YYYY-MM-DD hh:mm:ss') AS triggerDefaultStartConfOnNew,    
--to_char (cross_default_start_conf_on,  'YYYY-MM-DD hh:mm:ss') AS crossDefaultStartConfOn,  
trigger_default_class_1,      
trigger_default_rank,      
--cross_default_partition_key,
Cross_default_id,      
cross_default_cocunut, 
cross_default_entity_type, 
cross_default_entity_type_desc,     
cross_default_class_1,      
cross_booking_company,      
cross_default_rank,      
number_days,
trigger_default_start_conf_on,
cross_default_start_conf_on, 
case when 
 abs(sub1.trigger_default_rank - sub1.cross_default_rank) < 2       
and sub1.trigger_default_rank < sub1.cross_default_rank       
and abs(sub1.trigger_default_start_conf_on - sub1.cross_default_start_conf_on) > 4      
then 'ERROR'
else 'OK'
end as DQI_CHECK
from  sub1      
order by 1
""",
"""-- 4.2.2.3.40
select distinct s.UNIT,  
s.rzb_cdr_counterparty_code, 
t.entity_type, 
t.entity_type_desc,
s.rzb_cdr_counterparty_name, 
s.rzb_cdr_internal_rating, 
s.rzb_cdr_ead_pre_ccf,
s.rzb_cdr_ead_pre_ccf, 
s.cc, 
s.cc1,
s.DQI_CHECK
from(
with sub1 as (            
select rzb_cdr.booking_company as rzb_cdr_booking_company, 
rzb_cdr.counterparty_code as rzb_cdr_counterparty_code, 
rzb_cdr.counterparty_name as rzb_cdr_counterparty_name, 
rzb_cdr.internal_rating AS rzb_cdr_internal_rating, 
rzb_cdr.ead_pre_ccf as rzb_cdr_ead_pre_ccf, 
rzb_default.counterparty_code as cc, 
rzb_default1.counterparty_code as cc1,
rzb_default1.partition_key AS  rzb_default1_partition_key,
rzb_cdr.partition_key AS rzb_cdr_partition_key       
from             
 (select counterparty_code, counterparty_name, internal_rating, sum(ead_pre_ccf) as ead_pre_ccf, case when booking_company in ('ABADE','ABADEKG','ABURASH','ACHAKG','ACRKG','ADAGI','ADAMKG','ADIAKG','ADIBRE','AGAMEMKG','AGIOS','AKRISIOS','ALPHAHOL','ARCANA','AUSBET','AUSHAN','AUSLEAS','AUSLPEM','BKGL','BLUMBE','BLUMKG','CANOPA','CERES','CUPID','CWNRIL','DOROS','ETEOKLES','FEBRIS','GENO','HABITO','KIINTEI','KIIOYS','LENTIAKG','LYRA','MOERBY','NORDOY','ORESKG','OSTALB','PELIAS','PLANA','RAIFGGL','RAIFKGL','RANX','RANXIV','RAV','RCL','REALBETA',
'RIL','RILVII','RILXIV','RLAIR','RLAUXO','RLAVMUEN','RLBET','RLGRUND','RLIMMO','RLNORDAB','RLPROJFI','RRENT','RUBRA','SAMARA','SCHRUNS','SINIS','SOLARII','STYRIAIM','THYMO','UNTERINN','URSA','VINDOKG','WEGA', 'AGIOS','RCL','CWNRIL','SCHRUNS','PELIAS','REALBETA') then 'RLGMBH' else booking_company end booking_company, partition_key            
  from rzb_cdr             
  where ead_pre_ccf>0
  and default_flag='T'
  and booking_company not in ('AMYKOS', 'ARESN', 'ARUM', 'BAILE', 'BBCNS', 'BBCRS', 'BLS', 'CEHOL', 'CINOVA', 'CISHOL', 'DAVHOL', 'DDSTB', 'FLOR', 'GOLDRB', 'HYPOCM', 'IHSGMBH', 'IHSRBI', 'KATVV', 'KONEVOVA', 'LOTA', 'LSHG', 'OEVK', 'PCRRS', 'PERSES', 'POINTON',            
 'PRODEAL', 'PROOFF', 'RAIA', 'RALT', 'RALTOHG', 'RAMRO', 'RBDE', 'RBHUGMBH', 'RBIIA', 'RBILEA', 'RBIM', 'RBSPKHR', 'RBSPKMS', 'RBSPKRO', 'RBUK', 'RCPF', 'REBAUM', 'RECA', 'RFJ3', 'RFNL', 'RFNLRO', 'RIAG', 'RIAPL', 'RIBET', 'RIDICZ', 'RIDIRECT', 'RIDISK', 'RILG',             
'RILH', 'RINVEST', 'RISP', 'RKAG', 'RLBHOLD', 'RLI', 'RLKZ', 'RLLT', 'RLPLRF', 'RLRETE', 'RMOA', 'RNPF', 'ROOFDPR', 'ROOFPL', 'ROOFPL2', 'RPBRO', 'RPC', 'RPIC', 'RPPH', 'RSC', 'RSHR', 'RTC', 'RWBB', 'RZBINV', 'RZBPEAG', 'RZBPEGMB', 'RZBSEKTO', 'RZBVER', 'SALVE', 'SAMPROP',            
 'SCTHAV', 'SCTJ', 'SCTO2', 'SEEHOL', 'SRLBH', 'TAM', 'TGF', 'TGS', 'UPC', 'VINDALO', 'VORSORGE',  'VAJNOR', 'URSA', 'ACHAKG', 'CANOPA', 'CERES', 'ETEOKLES', 'LYRA', 'RANELF', 'RBL','RLAIR','RLFLU','ABADE','RLLAMDA')            
  and rating_model not in ('PI','Micro')
  group by counterparty_code, counterparty_name, internal_rating, booking_company    ,partition_key        
) rzb_cdr             
left outer join             
 (select distinct cocunut counterparty_code, case when booking_company in ('ABADE','ABADEKG','ABURASH','ACHAKG','ACRKG','ADAGI','ADAMKG','ADIAKG','ADIBRE','AGAMEMKG','AGIOS','AKRISIOS','ALPHAHOL','ARCANA','AUSBET','AUSHAN','AUSLEAS','AUSLPEM','BKGL','BLUMBE','BLUMKG','CANOPA','CERES','CUPID','CWNRIL','DOROS','ETEOKLES','FEBRIS','GENO','HABITO','KIINTEI','KIIOYS','LENTIAKG','LYRA','MOERBY','NORDOY','ORESKG','OSTALB','PELIAS','PLANA','RAIFGGL','RAIFKGL','RANX','RANXIV','RAV','RCL','REALBETA',
'RIL','RILVII','RILXIV','RLAIR','RLAUXO','RLAVMUEN','RLBET','RLGRUND','RLIMMO','RLNORDAB','RLPROJFI','RRENT','RUBRA','SAMARA','SCHRUNS','SINIS','SOLARII','STYRIAIM','THYMO','UNTERINN','URSA','VINDOKG','WEGA', 'AGIOS','RCL','CWNRIL','SCHRUNS','PELIAS','REALBETA') then 'RLGMBH' else booking_company end booking_company ,partition_key             
  from rzb_default             
  where default_status ='A'             
  and default_status_detail in ('P','E')             
  and default_classification_1 in ('D1','D2','D3','D4','D5','D6','D7','D8','D9','D10','D11','D12')
  ---and trunc(start_date) > to_date ('2016-01-01','yyyy-mm-dd') 
  --and rzb_default.partition_key      = to_char(to_date('#DQI_PROC_DATETIME#','DDmonYYYY'),'YYYYMMDD')||'0000'              
) rzb_default            
on rzb_cdr.counterparty_code = rzb_default.counterparty_code and rzb_cdr.booking_company =rzb_default.booking_company            
left outer join             
 (select distinct cocunut counterparty_code ,partition_key            
  from rzb_default             
  where default_status ='A'             
  and default_status_detail in ('P','E')
  and trunc(start_date) > to_date ('2016-01-01','yyyy-mm-dd')              
  and default_classification_1 in ('D1','D2','D3','D4','D5','D6','D7','D8','D9','D10','D11','D12')
  --and rzb_default.partition_key      = to_char(to_date('#DQI_PROC_DATETIME#','DDmonYYYY'),'YYYYMMDD')||'0000'              
) rzb_default1            
on rzb_cdr.counterparty_code = rzb_default1.counterparty_code            
)            
----------------            
-- DETAIL SQL --            
----------------            
select rzb_cdr_booking_company as UNIT,  
rzb_cdr_counterparty_code, 
rzb_cdr_counterparty_name, 
rzb_cdr_internal_rating, 
rzb_cdr_ead_pre_ccf, 
cc, 
cc1,
case when 
    sub1.cc is null  and  sub1.cc1  is not null 
    then 'ERROR'
    else 'OK'
    end as DQI_CHECK            
from      sub1      
--where sub1.rzb_cdr_partition_key  = to_char(to_date('#DQI_PROC_DATETIME#','DDmonYYYY'),'YYYYMMDD')||'0002'
order by 1) s
left join entity e on s.rzb_cdr_counterparty_code = e.entity_code
left join entity_type  t on t.entity_type= e.et_code_user
""",
"""-- 4.2.2.3.46
select 
d.booking_company AS UNIT,
e.entity_code as Counterparty_Code, 
e.entity_desc as Counterparty_Name, 
e.et_code_user as ctype_code,
t.entity_type_desc as CTYPE_NAME,
d.default_id as  Def_ID,
d.start_date as Default_Start_Date,
d.default_end_date, r.long_term_rating ,
d.start_date AS  CurrentStartDate,
d.default_end_date AS  DefaultEndDate,
case when 
 r.long_term_rating in  
 ('FIN_10', 'CORP_10', 'CORPL_10', 'CORPPOOL_10', 'FFC_5.0', 'FIN_D', 'FUND_D', 'INL_5.0', 'LFI_5.0', 'LRG_10_HU', 'LRG_10_RW', 'SLOT_6.5', 'SLOT_RE_6.5', 'SLOT_PF_6.5', 'SME_10', 'SOV_10')
then 'ERROR'
else 'OK'
end as DQI_CHECK
 -----d.partition_key
from rzb_default d 
left join issuer_credit_ratings r 
on d.cocunut = r.entity_code 
and d.partition_key = r.partition_key
left join entity e 
on d.cocunut=e.entity_code
and d.partition_key = e.partition_key
left join entity_type t 
on e.et_code_user = t.entity_type
and d.partition_key = e.partition_key
join (select cocunut, max(default_end_date) as default_end_date 
from rzb_default
-- where partition_key  = to_char(to_date('#DQI_PROC_DATETIME#','DDmonYYYY'),'YYYYMMDD')||'0000'
group by cocunut) d2 
on d.cocunut = d2.cocunut and d.default_end_date=d2.default_end_date
where d.cocunut not in (select cocunut from rzb_default where default_status_detail <> 'C' 
-- and partition_key  = to_char(to_date('#DQI_PROC_DATETIME#','DDmonYYYY'),'YYYYMMDD')||'0000'
)  
and d.default_end_cause = '0' 
and e.entity_code is not null
and trunc(d.start_date) > to_date ('2016-01-01','yyyy-mm-dd')
-- and d.partition_key  = to_char(to_date('#DQI_PROC_DATETIME#','DDmonYYYY'),'YYYYMMDD')||'0000'
order by UNIT
""",
"""-- 4.2.2.3.47
with defaulted_entities as (      
select      
   E.ENTITY_CODE  as COCUNUT      
   ,E.ET_CODE_USER as CTYPE_CODE
  ,T.ENTITY_TYPE_DESC as CTYPE_NAME
  ,i.long_term_rating as RATING      
  ,E.DEFAULT_FLAG   as DEFAULT_FLAG      
  , replace(E.IMPORT_SOURCE, 'GR_','') as  UNIT_CODE 
    , E.PARTITION_KEY as PARTITION_KEY
from ENTITY  E   
left join issuer_credit_ratings i on (E.ENTITY_CODE=i.entity_code and E.PARTITION_KEY = I.PARTITION_KEY )   
left join entity_type t on e.et_code_user = t.entity_type 
 where      
    i.agency_code='INTERNAL' and E.STATUS= 'A'  and E.DEFAULT_FLAG  = 'T' -- 1. check all active customers defaulted customers (using default_flag)     
)      
------------------      
-- DETAILED SQL --      
------------------      
select      
UNIT_CODE        
  ,COCUNUT   
  ,CTYPE_CODE
  ,CTYPE_NAME   
  ,RATING      
  ,DEFAULT_FLAG  
  ,CASE when RATING not in ('FIN_10', 'CORP_10', 'CORPL_10', 'CORPPOOL_10', 'FFC_5.0', 'FIN_D', 'FUND_D', 'INL_5.0', 'LFI_5.0', 'LRG_10_HU', 'LRG_10_RW', 'SLOT_6.5', 'SLOT_RE_6.5', 'SLOT_PF_6.5', 'SME_10', 'SOV_10') THEN 'NOK' ELSE 'OK' END DQI
from defaulted_entities      
  -- there is no non-cross default in a different entity
""",
"""-- 4.2.2.3.48
with customer_exposure as (      
  select     
     substr(BOOK_CODE, 1, instr(BOOK_CODE, '_')-1)          as ENTITY_ID        
    ,COCUNUT                                                as COCUNUT    
    ,sum(EXPOSURE)                                          as EXPOSURE      
  from (    
    -- Exposure per original customer in V_RZB_CDR    
    select    
       book_code                          as BOOK_CODE    
      ,counterparty_code                  as COCUNUT    
      ,round(ead_pre_ccf,0)               as EXPOSURE    
    from v_rzb_cdr             
    --    
    union all        
    -- Nominal value of underlyings of CDSs    
    select    
       cd.book_code                           as BOOK_CODE    
      ,b.issuer_code                          as COCUNUT    
      ,round(cd.nominal*c.exch_rate,0)        as EXPOSURE    
    from credit_derivative cd    
    left join reference_list rl     
      on cd.basket_reference = rl.basket_reference    
    left join bond b     
      on rl.contract_reference = b.security_reference    
    left join currency c     
      on cd.currency = c.ccy_code               
    --    
    union all            
    -- Nominal values of gaurantees    
    select           
       gua.book_code                                     as BOOK_CODE    
      ,gua.ISSUER_GUARANTOR                              as COCUNUT    
      ,gua.NOMINAL*c.exch_rate                           as EXPOSURE    
    from GUARANTEE gua            
    left join currency c     
      on     gua.currency       = c.ccy_code     
         and gua.partition_key  = c.partition_key        
  )    
  group by     
     substr(BOOK_CODE, 1, instr(BOOK_CODE, '_')-1)    
    ,COCUNUT           
)    
-- Join tables entity, customer_exposure and rzb_default in order to get exposure    
 -- and default information in one result.    
, def_cust_with_default_entry as (    
 SELECT     
     entity.ENTITY_CODE       as COCUNUT    
    ,expo.ENTITY_ID            as ENTITY_ID    
    ,expo.EXPOSURE             as EXPOSURE    
    ,entity.ATTRIBUTE_4       as INTERNAL_RATING        
    ,entity.ENTITY_CODE_2     as CUSTOMER_ID    
    ,d.DEFAULT_ID             as DEFAULT_UNID    
    ,d.START_DATE             as DEFAULT_START_DATE    
    ,d.DEFAULT_STATUS_DETAIL  as DETAILED_DEFAULT_STATUS      
    ,entity.ATTRIBUTE_7       as DEFAULTED_BEFORE_2005_IND
    ,case when  entity.ATTRIBUTE_4    in ('5.0','6.5','D','10') and expo.ENTITY_ID not in ('AMYKOS', 'ARESN', 'ARUM', 'BAILE', 'BBCNS', 'BBCRS', 'BLS', 'CEHOL', 'CINOVA', 'CISHOL', 'DAVHOL', 'DDSTB', 'FLOR', 'GOLDRB', 'HYPOCM', 'IHSGMBH', 'IHSRBI', 'KATVV', 'KONEVOVA', 'LOTA', 'LSHG', 'OEVK', 'PCRRS', 'PERSES', 'POINTON',  'PRODEAL', 'PROOFF', 'RAIA', 'RALT', 'RALTOHG', 'RAMRO', 'RBDE', 'RBHUGMBH', 'RBIIA', 'RBILEA', 'RBIM', 'RBSPKHR', 'RBSPKMS', 'RBSPKRO', 'RBUK', 'RCPF', 'REBAUM', 'RECA', 'RFJ3', 'RFNL', 'RFNLRO', 'RIAG', 'RIAPL', 'RIBET', 'RIDICZ', 'RIDIRECT', 'RIDISK', 'RILG', 'RILH', 'RINVEST', 'RISP', 'RKAG', 'RLBHOLD', 'RLI', 'RLKZ', 'RLLT', 'RLPLRF', 'RLRETE', 'RMOA', 'RNPF', 'ROOFDPR', 'ROOFPL', 'ROOFPL2', 'RPBRO', 'RPC', 'RPIC', 'RPPH', 'RSC', 'RSHR', 'RTC', 'RWBB', 'RZBINV', 'RZBPEAG', 'RZBPEGMB', 'RZBSEKTO', 'RZBVER', 'SALVE', 'SAMPROP', 'SCTHAV', 'SCTJ', 'SCTO2', 'SEEHOL', 'SRLBH', 'TAM', 'TGF', 'TGS', 'UPC', 'VINDALO', 'VORSORGE', 'VAJNOR', 'URSA', 'ACHAKG', 'CANOPA', 'CERES', 'ETEOKLES', 'LYRA', 'RANELF', 'RBL','RLAIR','RLFLU') then 'ERROR' else 'OK' end as DQI_CHECK
  from ENTITY entity      
  -- For every customer (cocunut), get exposure information    
  left outer join customer_exposure expo     
    on entity.ENTITY_CODE = expo.COCUNUT      
  -- for every customer (cocunut), get default entry    
  left outer join ( select DEFAULT_ID, COCUNUT, START_DATE, DEFAULT_STATUS_DETAIL, DEFAULT_STATUS, case when BOOKING_COMPANY ='RBI' then 'HO' else  BOOKING_COMPANY end  BOOKING_COMPANY    
from RZB_DEFAULT)  d    
    on    entity.ENTITY_CODE        = D.COCUNUT    
      and d.BOOKING_COMPANY         = expo.ENTITY_ID    
      and d.DEFAULT_STATUS_DETAIL    in ('P','E') -- 2. Customer with default rating 5.0, 6.5 or D    
      and d.DEFAULT_STATUS         = 'A'     -- 4. Only active records        
  where     
        entity.STATUS       = 'A' 
--and entity.partition_key =     to_char(to_date('#DQI_PROC_DATETIME#','DDmonYYYY'),'YYYYMMDD')||'0000'        
    and entity.ATTRIBUTE_7 is null -- 1. Consider only customer that have defaulted after the 1.1.2005.    
    and entity.ATTRIBUTE_4  in ('5.0','6.5','D','10') -- 2. Customer with default rating 5.0, 6.5    
    and expo.EXPOSURE > 0 -- 3. Consider only customers with exposure 
    and expo.ENTITY_ID not in ('AMYKOS', 'ARESN', 'ARUM', 'BAILE', 'BBCNS', 'BBCRS', 'BLS', 'CEHOL', 'CINOVA', 'CISHOL', 'DAVHOL', 'DDSTB', 'FLOR', 'GOLDRB', 'HYPOCM', 'IHSGMBH', 'IHSRBI', 'KATVV', 'KONEVOVA', 'LOTA', 'LSHG', 'OEVK', 'PCRRS', 'PERSES', 'POINTON',  'PRODEAL', 'PROOFF', 'RAIA', 'RALT', 'RALTOHG', 'RAMRO', 'RBDE', 'RBHUGMBH', 'RBIIA', 'RBILEA', 'RBIM', 'RBSPKHR', 'RBSPKMS', 'RBSPKRO', 'RBUK', 'RCPF', 'REBAUM', 'RECA', 'RFJ3', 'RFNL', 'RFNLRO', 'RIAG', 'RIAPL', 'RIBET', 'RIDICZ', 'RIDIRECT', 'RIDISK', 'RILG', 'RILH', 'RINVEST', 'RISP', 'RKAG', 'RLBHOLD', 'RLI', 'RLKZ', 'RLLT', 'RLPLRF', 'RLRETE', 'RMOA', 'RNPF', 'ROOFDPR', 'ROOFPL', 'ROOFPL2', 'RPBRO', 'RPC', 'RPIC', 'RPPH', 'RSC', 'RSHR', 'RTC', 'RWBB', 'RZBINV', 'RZBPEAG', 'RZBPEGMB', 'RZBSEKTO', 'RZBVER', 'SALVE', 'SAMPROP', 'SCTHAV', 'SCTJ', 'SCTO2', 'SEEHOL', 'SRLBH', 'TAM', 'TGF', 'TGS', 'UPC', 'VINDALO', 'VORSORGE', 'VAJNOR', 'URSA', 'ACHAKG', 'CANOPA', 'CERES', 'ETEOKLES', 'LYRA', 'RANELF', 'RBL','RLAIR','RLFLU')      
)
------------------    
-- DETAILED SQL --    
------------------    
select     
d.ENTITY_ID as UNIT_CODE         
,d.COCUNUT  
,e.et_code_user as ctype_code
,t.entity_type_desc as CTYPE_NAME
,d.EXPOSURE as EXPO_EXPOSURE
,d.INTERNAL_RATING        
,d.CUSTOMER_ID    
,d.DEFAULT_UNID    
,d.DEFAULT_START_DATE    
,d.DETAILED_DEFAULT_STATUS      
,d.DEFAULTED_BEFORE_2005_IND  
,d.DQI_CHECK  
from def_cust_with_default_entry d  
left join entity e on d.cocunut=e.entity_code
left join entity_type t on e.et_code_user = t.entity_type
where d.DEFAULT_UNID is null  -- default entry not found for the defaulted customer   
order by 1
""",
"""-- 4.2.2.1.19A
with 
t_0 as (
  select to_date(to_char(to_date('"""+rd[6:8]+rd[4:6]+"""2022','DDMMYYYY'),'YYYYMMDD'),'YYYYMMDD') as cut_off_date from dual 
)
, t_6 as (
  select cut_off_date - 182 - to_char(cut_off_date -182, 'DD') as cut_off_date from (select cut_off_date from t_0)
)
, t_12 as (
  select cut_off_date - 182*2 - to_char(cut_off_date -182*2, 'DD') as cut_off_date from (select cut_off_date from t_0)
)
, t_18 as (
  select cut_off_date - 182*3 - to_char(cut_off_date -182*3, 'DD') as cut_off_date from (select cut_off_date from t_0)
)
, t_24 as (
  select cut_off_date - 182*4 - to_char(cut_off_date -182*4, 'DD') as cut_off_date from (select cut_off_date from t_0)
)
-- List of entities with non-default threshold for exposure or turnover.
-- The default thresholds are:
--   SMB_EXPOSURE_THRESHOLD =   100 000
--   CO_EXPOSURE_THRESHOLD  = 1 500 000
--   TURNOVER_THRESHOLD     = 5 000 000
, rb_threshold_exceptions as (  
select 'RBAL' as ENTITY_ID, 109000 as SMB_EXPOSURE_THRESHOLD, 1642000 as CO_EXPOSURE_THRESHOLD, 5467000 as TURNOVER_THRESHOLD from dual union
select 'RLAL' as ENTITY_ID, 109000 as SMB_EXPOSURE_THRESHOLD, 1642000 as CO_EXPOSURE_THRESHOLD, 5467000 as TURNOVER_THRESHOLD from dual union 
select 'RBBG' as ENTITY_ID, 256000 as SMB_EXPOSURE_THRESHOLD, 1529000 as CO_EXPOSURE_THRESHOLD, 5098000 as TURNOVER_THRESHOLD from dual union
select 'RLBG' as ENTITY_ID, 256000 as SMB_EXPOSURE_THRESHOLD, 1529000 as CO_EXPOSURE_THRESHOLD, 5098000 as TURNOVER_THRESHOLD from dual union
select 'RBBY' as ENTITY_ID, 121000 as SMB_EXPOSURE_THRESHOLD, 1806000 as CO_EXPOSURE_THRESHOLD, 6006000 as TURNOVER_THRESHOLD from dual union
select 'RLBY1' as ENTITY_ID, 121000 as SMB_EXPOSURE_THRESHOLD, 1806000 as CO_EXPOSURE_THRESHOLD, 6006000 as TURNOVER_THRESHOLD from dual union
select 'RBBH' as ENTITY_ID, 100000 as SMB_EXPOSURE_THRESHOLD, 1498000 as CO_EXPOSURE_THRESHOLD, 2495000 as TURNOVER_THRESHOLD from dual union
select 'RLBH' as ENTITY_ID, 100000 as SMB_EXPOSURE_THRESHOLD, 1498000 as CO_EXPOSURE_THRESHOLD, 2495000 as TURNOVER_THRESHOLD from dual union
select 'RBCZ' as ENTITY_ID, 307000 as SMB_EXPOSURE_THRESHOLD, 2048000 as CO_EXPOSURE_THRESHOLD, 10241000 as TURNOVER_THRESHOLD from dual union
select 'RLCZ' as ENTITY_ID, 307000 as SMB_EXPOSURE_THRESHOLD, 2048000 as CO_EXPOSURE_THRESHOLD, 10241000 as TURNOVER_THRESHOLD from dual union
select 'RBHR' as ENTITY_ID, 205000 as SMB_EXPOSURE_THRESHOLD, 1542000 as CO_EXPOSURE_THRESHOLD, 5107000 as TURNOVER_THRESHOLD from dual union
select 'RLHR' as ENTITY_ID, 205000 as SMB_EXPOSURE_THRESHOLD, 1542000 as CO_EXPOSURE_THRESHOLD, 5107000 as TURNOVER_THRESHOLD from dual union
select 'RBHU' as ENTITY_ID, 247000 as SMB_EXPOSURE_THRESHOLD, 1238000 as CO_EXPOSURE_THRESHOLD, 4136000 as TURNOVER_THRESHOLD from dual union
select 'RLHU' as ENTITY_ID, 247000 as SMB_EXPOSURE_THRESHOLD, 1238000 as CO_EXPOSURE_THRESHOLD, 4136000 as TURNOVER_THRESHOLD from dual union
select 'RBKO' as ENTITY_ID, 100000 as SMB_EXPOSURE_THRESHOLD, 1520000 as CO_EXPOSURE_THRESHOLD, 5080000 as TURNOVER_THRESHOLD from dual union
select 'RLKO' as ENTITY_ID, 100000 as SMB_EXPOSURE_THRESHOLD, 1520000 as CO_EXPOSURE_THRESHOLD, 5080000 as TURNOVER_THRESHOLD from dual union
select 'RBPL' as ENTITY_ID, 292000 as SMB_EXPOSURE_THRESHOLD, 1458000 as CO_EXPOSURE_THRESHOLD, 6072000 as TURNOVER_THRESHOLD from dual union
select 'RBRO' as ENTITY_ID, 190000 as SMB_EXPOSURE_THRESHOLD, 1434000 as CO_EXPOSURE_THRESHOLD, 4752000 as TURNOVER_THRESHOLD from dual union
select 'RLRO' as ENTITY_ID, 190000 as SMB_EXPOSURE_THRESHOLD, 1434000 as CO_EXPOSURE_THRESHOLD, 4752000 as TURNOVER_THRESHOLD from dual union
select 'RBRU' as ENTITY_ID, 87000 as SMB_EXPOSURE_THRESHOLD, 2128000 as CO_EXPOSURE_THRESHOLD, 7092000 as TURNOVER_THRESHOLD from dual union
select 'RLRU' as ENTITY_ID, 87000 as SMB_EXPOSURE_THRESHOLD, 2128000 as CO_EXPOSURE_THRESHOLD, 7092000 as TURNOVER_THRESHOLD from dual union
select 'RBRS' as ENTITY_ID, 102000 as SMB_EXPOSURE_THRESHOLD, 1016000 as CO_EXPOSURE_THRESHOLD, 3039000 as TURNOVER_THRESHOLD from dual union
select 'RLRS' as ENTITY_ID, 102000 as SMB_EXPOSURE_THRESHOLD, 1016000 as CO_EXPOSURE_THRESHOLD, 3039000 as TURNOVER_THRESHOLD from dual union
select 'TBSK' as ENTITY_ID, 200000 as SMB_EXPOSURE_THRESHOLD, 1510000 as CO_EXPOSURE_THRESHOLD, 8060000 as TURNOVER_THRESHOLD from dual union
select 'TLSK' as ENTITY_ID, 200000 as SMB_EXPOSURE_THRESHOLD, 1510000 as CO_EXPOSURE_THRESHOLD, 8060000 as TURNOVER_THRESHOLD from dual union
select 'AVAL' as ENTITY_ID, 137000 as SMB_EXPOSURE_THRESHOLD, 915000 as CO_EXPOSURE_THRESHOLD, 2441000 as TURNOVER_THRESHOLD from dual union
select 'RLUA' as ENTITY_ID, 137000 as SMB_EXPOSURE_THRESHOLD, 915000 as CO_EXPOSURE_THRESHOLD, 2441000 as TURNOVER_THRESHOLD from dual union
select 'RBAL' as ENTITY_ID, 109000 as SMB_EXPOSURE_THRESHOLD, 1642000 as CO_EXPOSURE_THRESHOLD, 5467000 as TURNOVER_THRESHOLD from dual union
select 'RLAL' as ENTITY_ID, 109000 as SMB_EXPOSURE_THRESHOLD, 1642000 as CO_EXPOSURE_THRESHOLD, 5467000 as TURNOVER_THRESHOLD from dual union 
select 'RBBG' as ENTITY_ID, 256000 as SMB_EXPOSURE_THRESHOLD, 1529000 as CO_EXPOSURE_THRESHOLD, 5098000 as TURNOVER_THRESHOLD from dual union
select 'RLBG' as ENTITY_ID, 256000 as SMB_EXPOSURE_THRESHOLD, 1529000 as CO_EXPOSURE_THRESHOLD, 5098000 as TURNOVER_THRESHOLD from dual union
select 'RBBY' as ENTITY_ID, 121000 as SMB_EXPOSURE_THRESHOLD, 1806000 as CO_EXPOSURE_THRESHOLD, 6006000 as TURNOVER_THRESHOLD from dual union
select 'RLBY1' as ENTITY_ID, 121000 as SMB_EXPOSURE_THRESHOLD, 1806000 as CO_EXPOSURE_THRESHOLD, 6006000 as TURNOVER_THRESHOLD from dual union
select 'RBBH' as ENTITY_ID, 100000 as SMB_EXPOSURE_THRESHOLD, 1498000 as CO_EXPOSURE_THRESHOLD, 2495000 as TURNOVER_THRESHOLD from dual union
select 'RLBH' as ENTITY_ID, 100000 as SMB_EXPOSURE_THRESHOLD, 1498000 as CO_EXPOSURE_THRESHOLD, 2495000 as TURNOVER_THRESHOLD from dual union
select 'RBCZ' as ENTITY_ID, 307000 as SMB_EXPOSURE_THRESHOLD, 2048000 as CO_EXPOSURE_THRESHOLD, 10241000 as TURNOVER_THRESHOLD from dual union
select 'RLCZ' as ENTITY_ID, 307000 as SMB_EXPOSURE_THRESHOLD, 2048000 as CO_EXPOSURE_THRESHOLD, 10241000 as TURNOVER_THRESHOLD from dual union
select 'RBHR' as ENTITY_ID, 205000 as SMB_EXPOSURE_THRESHOLD, 1542000 as CO_EXPOSURE_THRESHOLD, 5107000 as TURNOVER_THRESHOLD from dual union
select 'RLHR' as ENTITY_ID, 205000 as SMB_EXPOSURE_THRESHOLD, 1542000 as CO_EXPOSURE_THRESHOLD, 5107000 as TURNOVER_THRESHOLD from dual union
select 'RBHU' as ENTITY_ID, 247000 as SMB_EXPOSURE_THRESHOLD, 1238000 as CO_EXPOSURE_THRESHOLD, 4136000 as TURNOVER_THRESHOLD from dual union
select 'RLHU' as ENTITY_ID, 247000 as SMB_EXPOSURE_THRESHOLD, 1238000 as CO_EXPOSURE_THRESHOLD, 4136000 as TURNOVER_THRESHOLD from dual union
select 'RBKO' as ENTITY_ID, 100000 as SMB_EXPOSURE_THRESHOLD, 1520000 as CO_EXPOSURE_THRESHOLD, 5080000 as TURNOVER_THRESHOLD from dual union
select 'RLKO' as ENTITY_ID, 100000 as SMB_EXPOSURE_THRESHOLD, 1520000 as CO_EXPOSURE_THRESHOLD, 5080000 as TURNOVER_THRESHOLD from dual union
select 'RBPL' as ENTITY_ID, 292000 as SMB_EXPOSURE_THRESHOLD, 1458000 as CO_EXPOSURE_THRESHOLD, 6072000 as TURNOVER_THRESHOLD from dual union
select 'RBRO' as ENTITY_ID, 190000 as SMB_EXPOSURE_THRESHOLD, 1434000 as CO_EXPOSURE_THRESHOLD, 4752000 as TURNOVER_THRESHOLD from dual union
select 'RLRO' as ENTITY_ID, 190000 as SMB_EXPOSURE_THRESHOLD, 1434000 as CO_EXPOSURE_THRESHOLD, 4752000 as TURNOVER_THRESHOLD from dual union
select 'RBRU' as ENTITY_ID, 87000 as SMB_EXPOSURE_THRESHOLD, 2128000 as CO_EXPOSURE_THRESHOLD, 7092000 as TURNOVER_THRESHOLD from dual union
select 'RLRU' as ENTITY_ID, 87000 as SMB_EXPOSURE_THRESHOLD, 2128000 as CO_EXPOSURE_THRESHOLD, 7092000 as TURNOVER_THRESHOLD from dual union
select 'RBRS' as ENTITY_ID, 102000 as SMB_EXPOSURE_THRESHOLD, 1016000 as CO_EXPOSURE_THRESHOLD, 3039000 as TURNOVER_THRESHOLD from dual union
select 'RLRS' as ENTITY_ID, 102000 as SMB_EXPOSURE_THRESHOLD, 1016000 as CO_EXPOSURE_THRESHOLD, 3039000 as TURNOVER_THRESHOLD from dual union
select 'TBSK' as ENTITY_ID, 200000 as SMB_EXPOSURE_THRESHOLD, 1510000 as CO_EXPOSURE_THRESHOLD, 8060000 as TURNOVER_THRESHOLD from dual union
select 'TLSK' as ENTITY_ID, 200000 as SMB_EXPOSURE_THRESHOLD, 1510000 as CO_EXPOSURE_THRESHOLD, 8060000 as TURNOVER_THRESHOLD from dual union
select 'AVAL' as ENTITY_ID, 137000 as SMB_EXPOSURE_THRESHOLD, 915000 as CO_EXPOSURE_THRESHOLD, 2441000 as TURNOVER_THRESHOLD from dual union
select 'RLUA' as ENTITY_ID, 137000 as SMB_EXPOSURE_THRESHOLD, 915000 as CO_EXPOSURE_THRESHOLD, 2441000 as TURNOVER_THRESHOLD from dual) 
-- List of entities with non-default threshold for exposure or turnover.
-- The default thresholds are:
--   SMB_EXPOSURE_THRESHOLD =   100 000
--   CO_EXPOSURE_THRESHOLD  = 1 500 000
--   TURNOVER_THRESHOLD     = 5 000 000
, rl_threshold_exceptions as (
  select 'XXX_DUMMY_XXX' as ENTITY_ID, 100000 as SMB_EXPOSURE_THRESHOLD, 1500000 as CO_EXPOSURE_THRESHOLD, 5000000 as TURNOVER_THRESHOLD from dual
)
-- This clause returns exsposure for each customer, reporting_date and entity_id including the exposure and turnover segmentation thresholds
, cust_exp_per_cutoff_thresholds as (
  select
       TO_DATE(substr(cdr.PARTITION_KEY,0,8),'YYYYMMDD')    as REPORTING_DATE
      ,cdr.COUNTERPARTY_CODE                                as COCUNUT
      ,substr(cdr.BOOK_CODE,1,instr(cdr.BOOK_CODE, '_')-1)  as ENTITY_ID
      ,round(sum(cdr.EAD_PRE_CCF),0)                        as EXPOSURE
      /*,min (case when substr(cdr.BOOK_CODE,1,2) = 'RB' then nvl(nwu_excp.TURNOVER_THRESHOLD,5000000)     -- take the defined threshold for NWU/NWL when avaialble, otherwise take the default one
                 when substr(cdr.BOOK_CODE,1,2) = 'RL' then nvl(nwl_excp.TURNOVER_THRESHOLD,5000000)
                 else 5000000 end)                          as TURNOVER_THRESHOLD
      ,min(case when substr(cdr.BOOK_CODE,1,2) = 'RB' then nvl(nwu_excp.CO_EXPOSURE_THRESHOLD,1500000)   -- take the defined threshold for NWU/NWL when avaialble, otherwise take the default one
                when substr(cdr.BOOK_CODE,1,2) = 'RL' then nvl(nwl_excp.CO_EXPOSURE_THRESHOLD,1500000)
                else 5000000 end)                           as CO_EXPOSURE_THRESHOLD*/
      ,min(nvl(nvl(nwu_excp.TURNOVER_THRESHOLD,nwl_excp.TURNOVER_THRESHOLD),5000000))           as TURNOVER_THRESHOLD
      ,min(nvl(nvl(nwu_excp.CO_EXPOSURE_THRESHOLD,nwl_excp.CO_EXPOSURE_THRESHOLD),1500000))     as CO_EXPOSURE_THRESHOLD
    from (
        v_rzb_cdr cdr
        left outer join rb_threshold_exceptions nwu_excp
          on nwu_excp.ENTITY_ID = substr(cdr.BOOK_CODE,1,instr(cdr.BOOK_CODE, '_')-1)       
        left outer join rl_threshold_exceptions nwl_excp
          on nwl_excp.ENTITY_ID = substr(cdr.BOOK_CODE,1,instr(cdr.BOOK_CODE, '_')-1) 
    )
    inner join (select t_0.cut_off_date as T0C, t_6.cut_off_date as T6C, t_12.cut_off_date as T12C ,t_18.cut_off_date as T18C, t_24.cut_off_date as T24C from t_0,t_6,t_12,t_18,t_24) 
      on TO_DATE(substr(cdr.PARTITION_KEY,0,8),'YYYYMMDD') in (T0C,T6C,T12C,T18C,T24C)   
   where
    RECORD_SOURCE != 'WCV'    -- 4. Record source should be != 'WCV'
   group by
      TO_DATE(substr(cdr.PARTITION_KEY,0,8),'YYYYMMDD')    -- cut-off day
     ,cdr.COUNTERPARTY_CODE                 -- cocunut
     ,substr(cdr.BOOK_CODE,1,instr(cdr.BOOK_CODE, '_')-1)  -- entity_id
)
--
-- This with clause joins cust_exp_per_cutoff_thresholds and entity, where the turnover will be taken. Minimum of exposure and turnover threshold per customer and reporting data is taken.
, cust_exp_turnover_minthr as (
select
     exp.ENTITY_ID                          as ENTITY_ID
    ,exp.REPORTING_DATE                  as REPORTING_DATE
    ,cust.ENTITY_CODE                     as COCUNUT
    ,cust.ET_CODE_USER                    as TIGER_ASSET_CLASS_ID
    ,sum(nvl(exp.EXPOSURE,0))             as EXPOSURE
   ,min(nvl(cust2.GCC_tas,0))  as TURNOVER                   --turnover should be the same per one cut-off
    ,min(exp.TURNOVER_THRESHOLD)          as MIN_TURNOVER_THRESHOLD
    ,min(exp.CO_EXPOSURE_THRESHOLD)       as MIN_CO_EXPOSURE_THRESHOLD
  from entity cust
  left outer join cust_exp_per_cutoff_thresholds exp
    on cust.entity_code = exp.cocunut
--new join for GCC head information
  left join (select e1.entity_code as child_code, e1.total_annual_sales as child_tas, e2.entity_code as GCC_code, e2.total_annual_sales as GCC_tas
                from esr s
                left join entity e1 on s.entity_code = e1.entity_code
                left join entity e2 on s.entity_group = e2.entity_code
             ) cust2
        on cust.entity_code = cust2.child_code
--       
  where
        cust.STATUS               = 'A'                 -- 1. Active customers
    AND cust.FINANCIALS_AVAILABLE = 'T'                   -- 3. Financials (balance sheet etc.) have to be available.
    AND cust.ET_CODE_USER         in ('6','29','41','63')          -- 2. Check only following SMB asset classes: ('6.'29','41','63')
    AND TO_DATE(substr(cust.partition_key,0,8),'YYYYMMDD') = exp.REPORTING_DATE
  group by 
     exp.ENTITY_ID
    ,exp.REPORTING_DATE
    ,ENTITY_CODE
    ,ET_CODE_USER   
)
------------------
-- DETAILED SQL --
------------------
select 
   ENTITY_ID    AS UNIT_CODE            
  ,COCUNUT                    as COCUNUT
  ,TIGER_ASSET_CLASS_ID       as TIGER_ASSET_CLASS_ID
  ,min(EXPOSURE)              as MIN_EXPOSURE
  ,min(MIN_CO_EXPOSURE_THRESHOLD)  as MIN_CO_EXPOSURE_THRESHOLD
  ,min(TURNOVER)              as MIN_TURNOVER
  ,min(MIN_TURNOVER_THRESHOLD)     as MIN_TURNOVER_THRESHOLD
from cust_exp_turnover_minthr
group by 
   ENTITY_ID 
  ,COCUNUT
  ,TIGER_ASSET_CLASS_ID
having
     min(EXPOSURE) > min(MIN_CO_EXPOSURE_THRESHOLD)
  OR min(TURNOVER) > min(MIN_TURNOVER_THRESHOLD)        
order by 1
""",
"""-- RI-GROUP.08.10.02.0017
select 
CASE
            WHEN ENTITY_CD ='HO' THEN 'RBI'
                        ELSE  ENTITY_CD
        END AS ENTITY_CD
        , DQI_CHECK
        ,CAST(ID AS VARCHAR(101)) AS ID
        ,EXPOSURE_AMT_EUR as CUMULATIVE_EXP_EUR
        ,EXP_AMT_ORIG as CUMULATIVE_EXP_ORIG
        ,COALESCE(EXP_CURRENCY,'N/A') AS EXP_CURRENCY
        ,EXP_ATTRIBUTE_5
        ,TOTAL_WCV_AMT_EUR as CUMULATIVE_WCV_EUR
,CASE
            WHEN EXPOSURE_AMT_EUR IS NULL OR TOTAL_WCV_AMT_EUR IS NULL THEN 'NOT OK'
            WHEN TOTAL_WCV_AMT_EUR <= EXPOSURE_AMT_EUR*1.015 THEN 'OK'
            ELSE 'NOT OK'
        END AS ERGEBNIS_0017
from
(
select 
CAST(LDE.CONTRACT_REFERENCE AS VARCHAR(101))             AS ID
,SUBSTR(LDE.BOOK_CODE, 0, INSTR(LDE.BOOK_CODE,'_')-1) AS ENTITY_CD
,CAST('LOANDEPO' AS VARCHAR(50))        AS DQI_CHECK
,CAST(lde.attribute_5  AS VARCHAR(100)) AS EXP_ATTRIBUTE_5
,lde.nominal AS EXP_AMT_ORIG
,CAST(LDE.CURRENCY   AS VARCHAR(3))   AS EXP_CURRENCY
,(LDE.NOMINAL * FX_LDE.EXCH_RATE)         AS EXPOSURE_AMT_EUR
,sum(CGU.AMOUNT_1 * FX_CGU.EXCH_RATE) AS TOTAL_WCV_AMT_EUR
from loandepo lde
JOIN  CURRENCY_EXCH_RATE FX_LDE
            ON FX_LDE.CCY_CODE = LDE.CURRENCY
            AND FX_LDE.CCY_SET_CODE LIKE'EZB_%'
and FX_LDE.PARTITION_KEY='"""+rd+"""0000'
join  contract_guarantee cgu
on CGU.BENEF_CONTRACT_REF = LDE.CONTRACT_REFERENCE
AND    CGU.BENEF_TABLE_NAME = 'LOANDEPO'
and CGU.PARTITION_KEY='"""+rd+"""0000'
JOIN  CURRENCY_EXCH_RATE FX_CGU
            ON FX_CGU.CCY_CODE = CGU.CURRENCY
            AND FX_CGU.CCY_SET_CODE LIKE'EZB_%'
and FX_CGU.PARTITION_KEY='"""+rd+"""0000'
AND (CGU.AMOUNT_1 * FX_CGU.EXCH_RATE) <>0
where (1=1)
and LDE.PARTITION_KEY='"""+rd+"""0000'
group by 
LDE.CONTRACT_REFERENCE,
LDE.BOOK_CODE,
lde.attribute_5,
lde.nominal,
LDE.CURRENCY,
(LDE.NOMINAL * FX_LDE.EXCH_RATE) 
UNION
select 
CAST(FAC.CONTRACT_REFERENCE AS VARCHAR(101))             AS ID
,SUBSTR(FAC.BOOK_CODE, 0, INSTR(FAC.BOOK_CODE,'_')-1) AS ENTITY_CD
 ,CAST('FACILITY' AS VARCHAR(50))        AS DQI_CHECK
,CAST(FAC.attribute_5  AS VARCHAR(100)) AS EXP_ATTRIBUTE_5
,FAC.AMOUNT AS EXP_AMT_ORIG
,CAST(FAC.CURRENCY   AS VARCHAR(3))   AS EXP_CURRENCY
,(FAC.AMOUNT * FX_FAC.EXCH_RATE)         AS EXPOSURE_AMT_EUR
,sum(CGU.AMOUNT_1 * FX_CGU.EXCH_RATE) AS TOTAL_WCV_AMT_EUR
from FACILITY FAC
JOIN  CURRENCY_EXCH_RATE FX_FAC
            ON FX_FAC.CCY_CODE = FAC.CURRENCY
            AND FX_FAC.CCY_SET_CODE LIKE'EZB_%'
and FX_FAC.PARTITION_KEY='"""+rd+"""0000'
join  contract_guarantee cgu
on CGU.BENEF_CONTRACT_REF = FAC.CONTRACT_REFERENCE
AND    CGU.BENEF_TABLE_NAME = 'FACILITY'
and CGU.PARTITION_KEY='"""+rd+"""0000'
JOIN  CURRENCY_EXCH_RATE FX_CGU
            ON FX_CGU.CCY_CODE = CGU.CURRENCY
            AND FX_CGU.CCY_SET_CODE LIKE'EZB_%'
and FX_CGU.PARTITION_KEY='"""+rd+"""0000'
AND (CGU.AMOUNT_1 * FX_CGU.EXCH_RATE) <>0
where (1=1)
and FAC.PARTITION_KEY='"""+rd+"""0000'
group by 
FAC.CONTRACT_REFERENCE,
FAC.BOOK_CODE,
FAC.attribute_5,
FAC.AMOUNT,
FAC.CURRENCY,
(FAC.AMOUNT * FX_FAC.EXCH_RATE) 
UNION
SELECT
CAST(ACC.ACCOUNT_REFERENCE AS VARCHAR(101)) AS ID
,SUBSTR(ACC.BOOK_CODE, 0, INSTR(ACC.BOOK_CODE,'_')-1) AS ENTITY_CD
,CAST('ACCOUNT' AS VARCHAR(50))         AS DQI_CHECK
,CAST(ACC.ATTRIBUTE_5 AS VARCHAR(100)) AS EXP_ATTRIBUTE_5
      ,(COALESCE(ACC.ACCRUALS,0)+ACC.BALANCE) AS EXP_AMT_ORIG
   ,CAST(ACC.CURRENCY AS VARCHAR(3))AS EXP_CURRENCY
   ,(COALESCE(ACC.ACCRUALS,0)+ACC.BALANCE) * FX_ACC.EXCH_RATE AS EXPOSURE_AMT_EUR
	,sum(CGU.AMOUNT_1 * FX_CGU.EXCH_RATE) AS TOTAL_WCV_AMT_EUR
   FROM ACCOUNT ACC
JOIN  CURRENCY_EXCH_RATE FX_ACC
            ON FX_ACC.CCY_CODE = ACC.CURRENCY
            AND FX_ACC.CCY_SET_CODE LIKE'EZB_%'
and FX_ACC.PARTITION_KEY='"""+rd+"""0000'
JOIN contract_guarantee CGU
ON ACC.ACCOUNT_REFERENCE = CGU.BENEF_CONTRACT_REF
AND    CGU.BENEF_TABLE_NAME = 'ACCOUNT'
and CGU.PARTITION_KEY='"""+rd+"""0000'
JOIN  CURRENCY_EXCH_RATE FX_CGU
            ON FX_CGU.CCY_CODE = CGU.CURRENCY
            AND FX_CGU.CCY_SET_CODE LIKE'EZB_%'
and FX_CGU.PARTITION_KEY='"""+rd+"""00000'
AND (CGU.AMOUNT_1 * FX_CGU.EXCH_RATE) <>0
where (1=1)
and ACC.PARTITION_KEY='"""+rd+"""0000'
group by 
ACC.ACCOUNT_REFERENCE ,
ACC.BOOK_CODE,
ACC.attribute_5,
(COALESCE(ACC.ACCRUALS,0)+ACC.BALANCE),
ACC.CURRENCY,
(COALESCE(ACC.ACCRUALS,0)+ACC.BALANCE) * FX_ACC.EXCH_RATE
UNION
select 
CAST(OOB.CONTRACT_REFERENCE AS VARCHAR(101))             AS ID
,SUBSTR(OOB.BOOK_CODE, 0, INSTR(OOB.BOOK_CODE,'_')-1) AS ENTITY_CD
 ,CAST('ON_OFF_BALANCE' AS VARCHAR(50))        AS DQI_CHECK
,CAST(OOB.attribute_5  AS VARCHAR(100)) AS EXP_ATTRIBUTE_5
,OOB.OUTSTANDING AS EXP_AMT_ORIG
,CAST(OOB.CURRENCY   AS VARCHAR(3))   AS EXP_CURRENCY
,(OOB.OUTSTANDING* FX_OOB.EXCH_RATE)         AS EXPOSURE_AMT_EUR
,sum(CGU.AMOUNT_1 * FX_CGU.EXCH_RATE) AS TOTAL_WCV_AMT_EUR
from ON_OFF_BALANCE OOB
JOIN  CURRENCY_EXCH_RATE FX_OOB
            ON FX_OOB.CCY_CODE = OOB.CURRENCY
            AND FX_OOB.CCY_SET_CODE LIKE'EZB_%'
and FX_OOB.PARTITION_KEY='"""+rd+"""0000'
join  contract_guarantee cgu
on CGU.BENEF_CONTRACT_REF = OOB.CONTRACT_REFERENCE
AND    CGU.BENEF_TABLE_NAME = 'ON_OFF_BALANCE'
and CGU.PARTITION_KEY='"""+rd+"""0000'
JOIN  CURRENCY_EXCH_RATE FX_CGU
            ON FX_CGU.CCY_CODE = CGU.CURRENCY
            AND FX_CGU.CCY_SET_CODE LIKE'EZB_%'
and FX_CGU.PARTITION_KEY='"""+rd+"""0000'
AND (CGU.AMOUNT_1 * FX_CGU.EXCH_RATE) <>0
where (1=1)
and OOB.PARTITION_KEY='"""+rd+"""0000'
group by 
OOB.CONTRACT_REFERENCE,
OOB.BOOK_CODE,
OOB.attribute_5,
OOB.OUTSTANDING,
OOB.CURRENCY,
(OOB.OUTSTANDING* FX_OOB.EXCH_RATE) 

UNION
select 
CAST(FX.CONTRACT_REFERENCE AS VARCHAR(101))             AS ID
,SUBSTR(FX.BOOK_CODE, 0, INSTR(FX.BOOK_CODE,'_')-1) AS ENTITY_CD
 ,CAST('FOREX' AS VARCHAR(50))        AS DQI_CHECK
,CAST(FX.attribute_5  AS VARCHAR(100)) AS EXP_ATTRIBUTE_5
,FX.PURCHASE_CCY_NOMINAL AS EXP_AMT_ORIG
,CAST(FX.PURCHASE_CCY   AS VARCHAR(3))   AS EXP_CURRENCY
,(FX.PURCHASE_CCY_NOMINAL* FX_FX.EXCH_RATE)         AS EXPOSURE_AMT_EUR
,sum(CGU.AMOUNT_1 * FX_CGU.EXCH_RATE) AS TOTAL_WCV_AMT_EUR
from FOREX FX
JOIN  CURRENCY_EXCH_RATE FX_FX
            ON FX_FX.CCY_CODE = FX.PURCHASE_CCY
            AND FX_FX.CCY_SET_CODE LIKE'EZB_%'
and FX_FX.PARTITION_KEY='"""+rd+"""0000'
join  contract_guarantee cgu
on CGU.BENEF_CONTRACT_REF = FX.CONTRACT_REFERENCE
AND    CGU.BENEF_TABLE_NAME = 'FOREX'
and CGU.PARTITION_KEY='"""+rd+"""0000'
JOIN  CURRENCY_EXCH_RATE FX_CGU
            ON FX_CGU.CCY_CODE = FX.PURCHASE_CCY
            AND FX_CGU.CCY_SET_CODE LIKE'EZB_%'
and FX_CGU.PARTITION_KEY='"""+rd+"""0000'
AND (CGU.AMOUNT_1 * FX_CGU.EXCH_RATE) <>0
where (1=1)
and FX.PARTITION_KEY='"""+rd+"""0000'
group by 
FX.CONTRACT_REFERENCE,
FX.BOOK_CODE,
FX.attribute_5,
FX.PURCHASE_CCY_NOMINAL,
FX.PURCHASE_CCY,
(FX.PURCHASE_CCY_NOMINAL* FX_FX.EXCH_RATE) 

UNION
select 
CAST(REC.CONTRACT_REFERENCE AS VARCHAR(101))             AS ID
,SUBSTR(REC.BOOK_CODE, 0, INSTR(REC.BOOK_CODE,'_')-1) AS ENTITY_CD
 ,CAST('RECEIVABLE' AS VARCHAR(50))        AS DQI_CHECK
,CAST(REC.attribute_5  AS VARCHAR(100)) AS EXP_ATTRIBUTE_5
,REC.nominal AS EXP_AMT_ORIG
,CAST(REC.CURRENCY   AS VARCHAR(3))   AS EXP_CURRENCY
,(REC.NOMINAL * FX_REC.EXCH_RATE)         AS EXPOSURE_AMT_EUR
,sum(CGU.AMOUNT_1 * FX_CGU.EXCH_RATE) AS TOTAL_WCV_AMT_EUR
from RECEIVABLE REC
JOIN  CURRENCY_EXCH_RATE FX_REC
            ON FX_REC.CCY_CODE = REC.CURRENCY
            AND FX_REC.CCY_SET_CODE LIKE'EZB_%'
and FX_REC.PARTITION_KEY='"""+rd+"""0000'
join  contract_guarantee cgu
on CGU.BENEF_CONTRACT_REF = REC.CONTRACT_REFERENCE
AND    CGU.BENEF_TABLE_NAME = 'RECEIVABLE'
and CGU.PARTITION_KEY='"""+rd+"""0000'
JOIN  CURRENCY_EXCH_RATE FX_CGU
            ON FX_CGU.CCY_CODE = CGU.CURRENCY
            AND FX_CGU.CCY_SET_CODE LIKE'EZB_%'
and FX_CGU.PARTITION_KEY='"""+rd+"""0000'
AND (CGU.AMOUNT_1 * FX_CGU.EXCH_RATE) <>0
where (1=1)
and REC.PARTITION_KEY='"""+rd+"""0000'
group by 
REC.CONTRACT_REFERENCE,
REC.BOOK_CODE,
REC.attribute_5,
REC.nominal,
REC.CURRENCY,
(REC.NOMINAL * FX_REC.EXCH_RATE) 
UNION
select 
CAST(DOC.CONTRACT_REFERENCE AS VARCHAR(101))             AS ID
,SUBSTR(DOC.BOOK_CODE, 0, INSTR(DOC.BOOK_CODE,'_')-1) AS ENTITY_CD
 ,CAST('DOC_CREDIT' AS VARCHAR(50))        AS DQI_CHECK
,CAST(DOC.attribute_5  AS VARCHAR(100)) AS EXP_ATTRIBUTE_5
,DOC.CONTRACT_AMOUNT AS EXP_AMT_ORIG
,CAST(DOC.CURRENCY_CODE   AS VARCHAR(3))   AS EXP_CURRENCY
,(DOC.CONTRACT_AMOUNT * FX_DOC.EXCH_RATE)         AS EXPOSURE_AMT_EUR
,sum(CGU.AMOUNT_1 * FX_CGU.EXCH_RATE) AS TOTAL_WCV_AMT_EUR
from DOC_CREDIT DOC
JOIN  CURRENCY_EXCH_RATE FX_DOC
            ON FX_DOC.CCY_CODE = DOC.CURRENCY_CODE
            AND FX_DOC.CCY_SET_CODE LIKE'EZB_%'
and FX_DOC.PARTITION_KEY='"""+rd+"""0000'
join  contract_guarantee cgu
on CGU.BENEF_CONTRACT_REF = DOC.CONTRACT_REFERENCE
AND    CGU.BENEF_TABLE_NAME = 'DOC_CREDIT'
and CGU.PARTITION_KEY='"""+rd+"""0000'
JOIN  CURRENCY_EXCH_RATE FX_CGU
            ON FX_CGU.CCY_CODE = CGU.CURRENCY
            AND FX_CGU.CCY_SET_CODE LIKE'EZB_%'
and FX_CGU.PARTITION_KEY='"""+rd+"""0000'
AND (CGU.AMOUNT_1 * FX_CGU.EXCH_RATE) <>0
where (1=1)
and DOC.PARTITION_KEY='"""+rd+"""0000'
group by 
DOC.CONTRACT_REFERENCE,
DOC.BOOK_CODE,
DOC.attribute_5,
DOC.CONTRACT_AMOUNT,
DOC.CURRENCY_CODE,
(DOC.CONTRACT_AMOUNT * FX_DOC.EXCH_RATE) 
)
where (1=1)
AND TOTAL_WCV_AMT_EUR <> 0.00
""",
"""-- RWA.014
with exposure as (
  select 
     cd.CONTRACT_REFERENCE                                  as UNID
    ,cd.ATTRIBUTE_5                                         as CDS_REF_ID
    ,cd.COUNTERPARTY                                      as CDS_COCUNUT  
    ,substr(cdr.BOOK_CODE,1,instr(cdr.BOOK_CODE, '_')-1)    as CDS_ENTITY_ID
    ,cd.NOMINAL                                             as CDS_NOMINAL_AMOUNT
    ,to_char(cd.VALUE_DATE)                               as CDS_START_DATE
    ,cd.LONG_SHORT                                          as CDS_LONG_SHORT_INDICATOR
    ,b.ISSUER_CODE                                          as UNDERLYING_ISSUER_COCUNUT
    ,e.INCORPORATION_COUNTRY                                as UNDERLYING_COUNTRY_CODE
    ,e.ET_CODE_USER                                         as UNDERL_ISSUER_ASSET_CLASS_ID
    ,sum(RW_EFF)/count(*)                                   as AVERAGE_RW
    ,sum(RWA)                                               as RWA_RELIEF
  from credit_derivative cd
  inner join reference_list rl 
    on cd.basket_reference = rl.basket_reference and substr(rl.partition_key,1,8) = substr(cd.partition_key,1,8)
  inner join (select * FROM security
        WHERE security_family = 'B') b 
    on rl.contract_reference = b.security_reference and substr(b.partition_key,1,8) = substr(cd.partition_key,1,8)
  inner join entity e 
    on b.issuer_code = e.entity_code and substr(e.partition_key,1,8) = substr(cd.partition_key,1,8)
  inner join v_rzb_cdr cdr
    on    cdr.contract_reference  = cd.contract_reference and substr(cdr.partition_key,1,8) = substr(cd.partition_key,1,8)
      and cdr.book_code           = cd.book_code
  where 
cd.partition_key = '"""+rd+"""0000'
AND e.ET_CODE_USER      in ('12','30','53','32','42','34') 
    AND cd.LONG_SHORT       = 'S'  
    AND cd.BOOK_CODE        like '%_B'  
    AND e.INCORPORATION_COUNTRY   in ('BE','BG','DK','DE','EE','FI','FR','GR','IE','IT','LV','LT','LU','MT','NL','AT','PL','PT','RO','SE','SK','SI','ES','CZ','HU','GB','CY')
  group by    
     cd.CONTRACT_REFERENCE
    ,cd.COUNTERPARTY
    ,cd.ATTRIBUTE_5
    ,substr(cdr.BOOK_CODE,1,instr(cdr.BOOK_CODE, '_')-1)
    ,cd.NOMINAL
    ,cd.VALUE_DATE
    ,cd.LONG_SHORT
    ,b.ISSUER_CODE
    ,e.INCORPORATION_COUNTRY
    ,e.ET_CODE_USER  
) 
select *
from exposure
""",
"""-- RWA.007
WITH DQM_RZB_CDR 
AS       
(
SELECT
 V_CDR.REPORTING_DATE AS VCDR_REPORTING_DATE,
 V_CDR.CONTRACT_REFERENCE AS VCDR_CONTRACT_REFERENCE,
 V_CDR.TABLE_NAME AS VCDR_TABLE_NAME,
 V_CDR.ATTRIBUTE_5 AS VCDR_ATTRIBUTE_5,
 V_CDR.BS_TYPE AS VCDR_BS_TYPE,
 V_CDR.BS_INDICATOR AS VCDR_BS_INDICATOR,
 V_CDR.CONTRACT_TYPE AS VCDR_CONTRACT_TYPE,
 NVL(V_CDR.COUNTERPARTY_NEW_CODE, V_CDR.COUNTERPARTY_CODE) AS VCDR_COUNTERPARTY_NEW_CODE,
 V_CDR.COUNTERPARTY_CODE AS VCDR_COUNTERPARTY_CODE,
 NVL(V_CDR.COUNTERPARTY_NEW_TYPE, V_CDR.COUNTERPARTY_TYPE) AS VCDR_COUNTERPARTY_NEW_TYPE,
 V_CDR.COUNTERPARTY_TYPE AS VCDR_COUNTERPARTY_TYPE,
 NVL(V_CDR.COUNTERPARTY_NEW_CATEGORY, V_CDR.COUNTERPARTY_CATEGORY) AS VCDR_COUNTERPARTY_NEW_CATEGORY,
 V_CDR.COUNTERPARTY_CATEGORY AS VCDR_COUNTERPARTY_CATEGORY,
 NVL(V_CDR.COUNTERPARTY_NEW_NAME, V_CDR.COUNTERPARTY_NAME) AS VCDR_COUNTERPARTY_NEW_NAME,
 V_CDR.COUNTERPARTY_NAME AS VCDR_COUNTERPARTY_NAME,
 NVL(V_CDR.BIS_ENTITY_TYPE_NEW, V_CDR.BIS_ENTITY_TYPE_ORIGINAL) AS VCDR_BIS_ENTITY_TYPE,
 NVL(V_CDR.GCC_HEAD_NEW_CODE, V_CDR.GCC_HEAD_CODE) AS VCDR_GCC_HEAD_NEW_CODE,
 V_CDR.GCC_HEAD_CODE AS VCDR_GCC_HEAD_CODE,
 NVL(V_CDR.GCC_HEAD_NEW_NAME, V_CDR.GCC_HEAD_NAME) AS VCDR_GCC_HEAD_NEW_NAME,
 V_CDR.GCC_HEAD_NAME AS VCDR_GCC_HEAD_NAME,
 V_CDR.GCC_HEAD_RATING AS VCDR_GCC_HEAD_RATING,
 NVL(V_CDR.INTERNAL_RATING_NEW, V_CDR.INTERNAL_RATING) AS VCDR_INTERNAL_RATING_NEW,
 V_CDR.INTERNAL_RATING AS VCDR_INTERNAL_RATING,
 V_CDR.RATING_MODEL AS VCDR_RATING_MODEL,
 V_CDR.EAD_PRE_CCF AS VCDR_EAD_PRE_CCF,
 V_CDR.EAD_PRE_CCF_SECURED AS VCDR_EAD_PRE_CCF_SECURED,
 V_CDR.EAD_PRE_CCF_UNSECURED AS VCDR_EAD_PRE_CCF_UNSECURED,
 NVL(V_CDR.METHOD_NEW, V_CDR.METHOD_ORIGINAL) AS VCDR_METHOD,
 V_CDR.BOOK_CODE AS VCDR_BOOK_CODE,
 V_CDR.CCF AS VCDR_CCF,
 V_CDR.CURRENCY AS VCDR_CURRENCY,
 V_CDR.DEFAULT_FLAG AS VCDR_DEFAULT_FLAG,
 V_CDR.EAD AS VCDR_EAD,
 V_CDR.COLLATERAL_TYPE AS VCDR_COLLATERAL_TYPE,
 V_CDR.FACILITY_REFERENCE AS VCDR_FACILITY_REFERENCE,
 V_CDR.LGD AS VCDR_LGD,
 V_CDR.PD AS VCDR_PD,
 V_CDR.RW_EFF AS VCDR_RW_EFF,
 V_CDR.RWA AS VCDR_RWA,
 BCL.ASSIGNED_AMOUNT AS BCL_ASSIGNED_AMOUNT,
 BCL.BENEF_MATURITY AS BCL_BENEF_MATURITY,
 BCL.BENEFICIARY_CONTRACT_REF AS BCL_BENEFICIARY_CONTRACT_REF,
 BCL.BENEFICIARY_CURRENCY AS BCL_BENEFICIARY_CURRENCY,
 BCL.BENEFICIARY_PRIORITY AS BCL_BENEFICIARY_PRIORITY,
 BCL.BENEFICIARY_SUB_REFERENCE AS BCL_BENEFICIARY_SUB_REFERENCE,
 BCL.BENEFICIARY_TABLE_NAME AS BCL_BENEFICIARY_TABLE_NAME,
 BCL.BS_TYPE AS BCL_BS_TYPE,
 BCL.COL_CATEGORY AS BCL_COL_CATEGORY,
 BCL.COL_FAMILY AS BCL_COL_FAMILY,
 BCL.COL_TYPE AS BCL_COL_TYPE,
 BCL.COV_LGD AS BCL_COV_LGD,
 BCL.COVERED_EAD AS BCL_COVERED_EAD,
 BCL.COVERED_EAD_PRE_CCF AS BCL_COVERED_EAD_PRE_CCF,
 BCL.CRM_ELIGIBILITY AS BCL_CRM_ELIGIBILITY,
 BCL.CRM_ID AS BCL_CRM_ID,
 BCL.CRM_METHOD AS BCL_CRM_METHOD,
 BCL.CSR_ID AS BCL_CSR_ID,
 BCL.FG_MORTGAGE_ALTERNATIVE AS BCL_FG_MORTGAGE_ALTERNATIVE,
 BCL.FLAG_TRIGGER_BEFORE AS BCL_FLAG_TRIGGER_BEFORE,
 BCL.FLOOR2 AS BCL_FLOOR2,
 BCL.GUAR_CONTRACT_REF AS BCL_GUAR_CONTRACT_REF,
 BCL.GUAR_CONTRACT_TYPE AS BCL_GUAR_CONTRACT_TYPE,
 BCL.GUAR_CURRENCY AS BCL_GUAR_CURRENCY,
 BCL.GUAR_MATURITY AS BCL_GUAR_MATURITY,
 BCL.GUAR_PRIORITY AS BCL_GUAR_PRIORITY,
 BCL.HCR AS BCL_HCR,
 BCL.HFX AS BCL_HFX,
 BCL.HR AS BCL_HR,
 BCL.LINK_MODE AS BCL_LINK_MODE,
 BCL.MATURITY_MISMATCH AS BCL_MATURITY_MISMATCH,
 BCL.PARTITION_KEY AS BCL_PARTITION_KEY,
 BCL.PARTITION_KEY_DATA AS BCL_PARTITION_KEY_DATA,
 BCL.PD_GUAR AS BCL_PD_GUAR,
 BCL.RISK_TYPE_MODE AS BCL_RISK_TYPE_MODE,
 BCL.RVCA AS BCL_RVCA,
 BCL.RW AS BCL_RW
FROM V_RZB_CDR V_CDR
LEFT JOIN
(SELECT ALLOCATION_ORDER, ASSIGNED_AMOUNT, BENEFICIARY_TABLE_NAME, 
BENEFICIARY_CONTRACT_REF, BENEF_MATURITY, BENEFICIARY_PRIORITY, BENEFICIARY_SUB_REFERENCE, 
BENEFICIARY_CURRENCY, BS_TYPE, CHECK_ERROR_NUMBER, COL_CATEGORY, COL_FAMILY, COL_TYPE, COV_LGD, COVERED_EAD, 
COVERED_EAD_PRE_CCF, CRM_ELIGIBILITY, CRM_ID, CRM_METHOD, CSR_ID, FG_MORTGAGE_ALTERNATIVE, FLAG_TRIGGER_BEFORE, FLOOR2, 
GUAR_CONTRACT_REF, GUAR_CONTRACT_TYPE, GUAR_CURRENCY, GUAR_MATURITY, GUAR_PRIORITY, HCR, HFX, HR, LINK_MODE, MATURITY_MISMATCH, 
PARTITION_KEY, PARTITION_KEY_DATA, PD_GUAR, RISK_TYPE_MODE, RVCA, RW, SCENARIO_ID 
FROM BIS_COLLATERAL_LINKS
WHERE CRM_METHOD <> 'STD'
/*NEW*/AND partition_key = '202111300194' /*0194 is the code of workspace RBI_GRP_FMA_GRP_UNCONS*/
      ) BCL 
ON V_CDR.CONTRACT_REFERENCE = BCL.BENEFICIARY_CONTRACT_REF
 AND V_CDR.TABLE_NAME = BCL.BENEFICIARY_TABLE_NAME
 AND V_CDR.COLLATERAL_TYPE = BCL.GUAR_CONTRACT_TYPE
 AND V_CDR.RESIDUAL_MATURITY = BCL.BENEF_MATURITY
 AND V_CDR.CURRENCY = BCL.BENEFICIARY_CURRENCY
 AND V_CDR.RECORD_SOURCE='T_CDR'
WHERE NVL(V_CDR.METHOD_NEW, V_CDR.METHOD_ORIGINAL) = 'FOU'
/*AND  V_CDR.REPORTING_DATE = to_date('#DQI_PROC_DATETIME#','DDmonYYYY')*/
/*NEW*/AND V_CDR.partition_key = '202111300194' /*0194 is the code of workspace RBI_GRP_FMA_GRP_UNCONS*/
ORDER BY V_CDR.CONTRACT_REFERENCE
)
SELECT 
       OB.UNIT, 
       OB.ATTRIBUTE_5,
       OFB_CDR.VCDR_RWA AS RWA_RELIEF,
       SEC_RWA.RWA_SECURED + OFB_CDR.VCDR_RWA AS RWA,
       GUA.ISSUER_GUARANTOR, 
       OB.COUNTERPARTY_CODE, 
       OB.MATURITY_DATE AS M_DATE_OB_C, 
       GUA.MATURITY_DATE AS M_DATE_GUA, 
       OB.TRADE_DATE AS T_DATE_OB, 
       GUA.TRADE_DATE AS T_DATE_GUA, 
       to_char(OB.MATURITY_DATE) AS MAT_DATE_OB, 
       to_char(GUA.MATURITY_DATE) AS MAT_DATE_GUA, 
       to_char(OB.TRADE_DATE) AS TRADE_DATE_OB, 
       to_char(GUA.TRADE_DATE)AS TRADE_DATE_GUA, 
       OB.NOMINAL AS N_OA, 
       GUA.NOMINAL AS N_GUA,
       OB.FINAL_BENEFICIARY
FROM 
(SELECT ATTRIBUTE_5, BENEFICIARY, BOOK_CODE, 
CONTRACT_REFERENCE, CONTRACT_TYPE, CURRENCY, DEAL_STATUS, DELETED, 
FLAG_GENERIC, GUARANTEE_PERCENTAGE, INTERMEDIATE_GUARANTOR, ISSUER_GUARANTOR, 
MATURITY_DATE, NOMINAL, SENIORITY, TRADE_DATE 
      FROM GUARANTEE
       /*NEW*/WHERE substr(partition_key,1,8) = to_char(to_date('"""+rd[6:8]+rd[4:6]+"""2022','DDMMYYYY'),'YYYYMMDD')) GUA
INNER JOIN 
(
SELECT SUBSTR(BOOK_CODE, 1, INSTR(BOOK_CODE, '_', 1)-1) AS UNIT, ATTRIBUTE_5, 
BOOK_CODE, CONTRACT_REFERENCE, CONTRACT_TYPE, COUNTERPARTY_CODE, 
DEAL_STATUS, FINAL_BENEFICIARY, MATURITY_DATE, TRADE_DATE, OUTSTANDING, NOMINAL 
            FROM ON_OFF_BALANCE 
            /*NEW*/WHERE substr(partition_key,1,8) = to_char(to_date('"""+rd[6:8]+rd[4:6]+"""2022','DDMMYYYY'),'YYYYMMDD')) 
   OB 
   ON GUA.ISSUER_GUARANTOR = OB.COUNTERPARTY_CODE AND OB.MATURITY_DATE = GUA.MATURITY_DATE AND OB.TRADE_DATE = GUA.TRADE_DATE AND OB.NOMINAL = GUA.NOMINAL
 /*NEW*/INNER JOIN (select * from CONSOLIDATION_PERIMETER WHERE substr(partition_key,1,8) = to_char(to_date('"""+rd[6:8]+rd[4:6]+"""2022','DDMMYYYY'),'YYYYMMDD')) CP ON OB.COUNTERPARTY_CODE = CP.ENTITY_CODE
INNER JOIN (
SELECT DISTINCT VCDR_PD, VCDR_RWA, VCDR_REPORTING_DATE, VCDR_CONTRACT_REFERENCE, VCDR_TABLE_NAME, 
VCDR_ATTRIBUTE_5, VCDR_BS_TYPE, VCDR_BS_INDICATOR, VCDR_CONTRACT_TYPE, VCDR_COUNTERPARTY_NEW_CODE, 
VCDR_COUNTERPARTY_NEW_TYPE, VCDR_COUNTERPARTY_NEW_CATEGORY, VCDR_COUNTERPARTY_NEW_NAME, VCDR_BIS_ENTITY_TYPE, 
VCDR_GCC_HEAD_CODE, VCDR_GCC_HEAD_NAME, VCDR_GCC_HEAD_RATING, VCDR_INTERNAL_RATING_NEW, VCDR_RATING_MODEL, 
VCDR_EAD_PRE_CCF, VCDR_EAD_PRE_CCF_SECURED, VCDR_EAD_PRE_CCF_UNSECURED, VCDR_METHOD, VCDR_BOOK_CODE, VCDR_CCF, 
VCDR_CURRENCY, VCDR_DEFAULT_FLAG, VCDR_EAD, VCDR_COLLATERAL_TYPE, VCDR_FACILITY_REFERENCE, VCDR_LGD, VCDR_RW_EFF 
FROM DQM_RZB_CDR
)
OFB_CDR ON OB.CONTRACT_REFERENCE = OFB_CDR.VCDR_CONTRACT_REFERENCE
INNER JOIN 
(SELECT SUM(VCDR_RWA) AS RWA_SECURED, BCL_GUAR_CONTRACT_REF AS GUAR_CONTRACT_REF
            FROM DQM_RZB_CDR
            WHERE VCDR_REPORTING_DATE = to_date('"""+rd[6:8]+rd[4:6]+"""2022','DDMMYYYY')
     GROUP BY BCL_GUAR_CONTRACT_REF ) 
  SEC_RWA ON SEC_RWA.GUAR_CONTRACT_REF = GUA.CONTRACT_REFERENCE
ORDER BY OB.UNIT
""",
"""-- RWA.015
with cds as (
select a.*, b.min_comp, b.max_comp, b.nominal as gua_nominal, b.rwa as GUA_RWA,
nvl(a.rwa,0) + nvl(b.rwa,0) as total_rwa_group,
least(nvl(a.rwa,0), nvl(b.rwa, 0)) + decode(a.crm, 'DRM', 0, (a.pd-a.pd_crm)/a.pd*a.rwa) as RWA_RELIEF
from
(select short.*, longc.nominal as nominal_long, longc.crm, longc.pd_crm from 
(select a.counterparty_code, a.currency, a.maturity_date, sum(a.ead_pre_ccf / a.exchange_rate) as nominal, sum(rwa) as  rwa, min(pd_used/100) as pd, min(rw_eff) as rw
from rzb_cdr a where a.table_name = 'CREDIT_DERIVATIVE' and a.booking_company = 'RBI' and a.record_source = 'T_CDR' and (a.collateral_type <> 'DRM' or a.collateral_type is null) and a.rwa > 0 and a.partition_key = '"""+rd+"""0194'
group by a.counterparty_code, a.currency, a.maturity_date) short,
(
select counterparty_code, currency, maturity_date, sum(nominal) as nominal, min(crm) as crm, max(pd_crm) as pd_crm from
(
select substr(c.contract_reference, 5, length(c.contract_reference)-6) as counterparty_code, b.currency as currency,
b.maturity_date as maturity_date, b.nominal, b.CONTRACT_TYPE as crm, pd_tab.pd as pd_crm
from credit_derivative b, reference_list c,
(select 
     r.ENTITY_CODE  as COCUNUT
    ,pd.PD          as PD
  from  ESR r
  inner join  BIS_LOCAL_PD_HANDLING pd
 on nvl(r.INTERNAL_RATING, r.RATING_CODE) = pd.INTERNAL_RATING     --take internal rating if available, otherwise the external one
       and AGENCY_CODE   = 'INTERNAL'         -- for simplicity, only internal ratings are considered, even though the CSD bought protection counterparty may have and external rating
       and SUPERVISOR_CODE   = (select max(SUPERVISOR_CODE) from BIS_LOCAL_PD_HANDLING where SUPERVISOR_CODE like 'FMA%')) pd_tab
where b.long_short = 'L' and b.contract_reference = c.basket_reference and b.attribute_5 not like '%*COL'
and pd_tab.cocunut = b.counterparty
union
select counterparty_code, currency, maturity_date, nominal, crm,  pd_crm from 
(select  b.contract_reference, b.attribute_5, b.counterparty, b.book_code,
substr(c.contract_reference, 5, length(c.contract_reference)-6) as counterparty_code, b.currency,
b.maturity_date, max(b.nominal) as nominal, min(b.CONTRACT_TYPE) as crm, max(pd_tab.pd) as pd_crm
from credit_derivative b, reference_list c,
(select 
     r.ENTITY_CODE  as COCUNUT
    ,pd.PD          as PD
  from  ESR r
  inner join  BIS_LOCAL_PD_HANDLING pd
 on nvl(r.INTERNAL_RATING, r.RATING_CODE) = pd.INTERNAL_RATING     --take internal rating if available, otherwise the external one
       and AGENCY_CODE   = 'INTERNAL'         -- for simplicity, only internal ratings are considered, even though the CSD bought protection counterparty may have and external rating
       and SUPERVISOR_CODE   = (select max(SUPERVISOR_CODE) from BIS_LOCAL_PD_HANDLING where SUPERVISOR_CODE like 'FMA%')) pd_tab
where b.long_short = 'L' and
c.basket_reference in (select x.benef_contract_ref from contract_guarantee x where x.guar_contract_ref = b.contract_reference) 
and b.attribute_5 like '%*COL'
and pd_tab.cocunut = b.counterparty
group by  b.contract_reference, b.attribute_5, b.counterparty, b.book_code, c.contract_reference, b.currency, b.maturity_date 
)
)
group by counterparty_code, currency, maturity_date
) longc
where longc.counterparty_code = short.counterparty_code 
and longc.currency = short.currency 
and longc.maturity_date = short.maturity_date) a,
( select min(booking_company) as min_comp, max(booking_company) as max_comp, x.counterparty_code, x.maturity_date, x.currency, sum(x.ead_pre_ccf / x.exchange_rate) as nominal, sum(x.rwa) as rwa
from rzb_cdr x
where x.booking_company  <> 'RBI' and x.record_source <> 'WCV' and x.partition_key = '"""+rd+"""0194' and product_group in ('GUARANTEE ISSUED', 'CREDIT_DERIVATIVE')
group by x.counterparty_code, x.maturity_date, x.currency) b
where 
a.counterparty_code = b.counterparty_code (+) and
a.maturity_date = b.maturity_date (+) and
a.currency = b.currency (+)
)
/*Detailed SQL*/
select * from cds
""",
"""-- RWA.016
with cds as (
  select  
    cd.PARTITION_KEY               as PARTITION_KEY
   ,cd.ATTRIBUTE_5                 as REF_ID
   ,to_char(cd.MATURITY_DATE)    as MATURITY_DATE
   ,cd.NOMINAL*c.exch_rate         as NOMINAL_AMOUNT
   ,cd.COUNTERPARTY                as COUNTERPARTY_COCUNUT
   ,cd.LONG_SHORT                  as LONG_SHORT_INDICATOR
   ,cd.CONTRACT_TYPE               as GPC
   ,b.SECURITY_REFERENCE           as UNDERLYING_REFERENCE
   ,cd.BOOK_CODE                   as BOOK_CODE
   ,sec.ISSUER_CODE                  as UNDERL_COCUNUT
  from (select * from CREDIT_DERIVATIVE where partition_key = '"""+rd+"""0000') cd
  inner join (select * from REFERENCE_LIST where partition_key = '"""+rd+"""0000') rl on cd.BASKET_REFERENCE = rl.BASKET_REFERENCE
  inner join (select * from security_positions where partition_key = '"""+rd+"""0000') b on rl.CONTRACT_REFERENCE = b.SECURITY_REFERENCE
  inner join (select * from security where nvl(security_family,0) = 'B' and partition_key = '"""+rd+"""0000') sec on b.security_reference = sec.security_reference
  left join (select * from currency where partition_key = '"""+rd+"""0000') c  on cd.currency      = c.ccy_code        
--  
  where 
    cd.LONG_SHORT = 'L'
)
--
, deals_exposure as (
  select     
     cdr.attribute_5               as DEAL_REF_ID
    ,cdr.COUNTERPARTY_CODE         as DEAL_COCUNUT
    ,sum(cdr.PD_USED)/count(*)/100 as DEAL_PD -- PD in percentace, tehrefore devided by 100
    ,sum(cdr.RW_EFF)/count(*)      as DEAL_RW
    ,sum(cdr.ead_pre_ccf)          as DEAL_EXPOSURE
    ,sum(cdr.RWA)                  as DEAL_RWA   
  from (select * from rzb_cdr where partition_key = '"""+rd+"""0194') cdr    
  inner join cds cds
    on cds.UNDERL_COCUNUT = cdr.COUNTERPARTY_CODE    
  group by     
   cdr.attribute_5
  ,cdr.COUNTERPARTY_CODE
)
--
-- get PD (probability of default) for every customer based on the rating
, pd_long as (
  select 
     r.ENTITY_CODE  as COCUNUT
    ,pd.PD          as PD
  from  (select * from ESR where partition_key = '"""+rd+"""0194') r
  inner join  (select * from BIS_LOCAL_PD_HANDLING where partition_key = '"""+rd+"""0194') pd
 on nvl(r.internal_rating, r.rating_code) = pd.INTERNAL_RATING     --take internal rating if available, otherwise the external one
       and AGENCY_CODE   = 'INTERNAL'         -- for simplicity, only internal ratings are considered, even though the CSD bought protection counterparty may have and external rating
       and SUPERVISOR_CODE   = (select max(SUPERVISOR_CODE) from BIS_LOCAL_PD_HANDLING where SUPERVISOR_CODE like 'FMA%')  -- 'FMA{max year}' e.g.'FMA2011'
)
------------------
------------------
-- Fro each CDS get all deals of a underlying customers - RWA, PD, EAD etc. 
-- Order they deals for each underlying customer by PD descendant. For each deal 
-- caluclate AVAILALBE nominal. The CSD nominal is distributed to the 
-- deals - first to to the deals with higher PD then with the lower one.
-- This approach is simulateing the distribution of guaranteed amount among 
-- the deals. The availalbe amount is calculated as NOMINAL - sum of exposure 
-- of all deals where a fraction of the nominal amount was already 
-- distributed (assigned).
, cds_and_deals as (
  select 
     cds.*
    ,deals.*  
    ,case when
           cds.NOMINAL_AMOUNT - (sum(DEAL_EXPOSURE) over (partition by cds.REF_ID order by DEAL_PD, DEAL_REF_ID)) + DEAL_EXPOSURE  >= 0
          then
           cds.NOMINAL_AMOUNT - (sum(DEAL_EXPOSURE) over (partition by cds.REF_ID order by DEAL_PD, DEAL_REF_ID)) + DEAL_EXPOSURE
          else
            0
          end      as AVAILABLE_NOMINAL
    ,pd_cds.PD as PD_CDS 
  --20
  from cds 
  -- get PD, RW and RWA of deals related to the cds underlying customer
  left outer join deals_exposure deals 
    on deals.DEAL_COCUNUT = cds.UNDERL_COCUNUT       
  -- get the PD of the CDS
  left outer join pd_long pd_cds
    on pd_cds.COCUNUT = cds.COUNTERPARTY_COCUNUT
)
--
-- For each CDS calculate RWA relief from the data gathered so far
, cds_rwa_relief as (
 select
     cds.*
 ,case when GPC != 'DRM' then 'ERROR' else 'OK' end as DQI_CHECK
    ,case when DEAL_EXPOSURE < AVAILABLE_NOMINAL then DEAL_EXPOSURE else AVAILABLE_NOMINAL end as EXPOSURE_FACTOR
    ,case when
        nvl((DEAL_PD - PD_CDS)/DEAL_PD*DEAL_RW*
         (case when DEAL_EXPOSURE < AVAILABLE_NOMINAL then DEAL_EXPOSURE else AVAILABLE_NOMINAL end),0) > 0
      then nvl((DEAL_PD - PD_CDS)/DEAL_PD*DEAL_RW*
         (case when DEAL_EXPOSURE < AVAILABLE_NOMINAL then DEAL_EXPOSURE else AVAILABLE_NOMINAL end),0) 
      else 0
      end as RWA_RELIEF
  from cds_and_deals cds
)
--
------------------
-- DETAILED SQL --
------------------
select      
 *
from cds_rwa_relief cds
"""
]

save_file(file_names, ex_folder, ws_nr, sqls)

Trying 4.2.2.1.45
Trying 4.2.2.3.38
Trying 4.2.2.3.39
Trying 4.2.2.3.40
Trying 4.2.2.3.46
Trying 4.2.2.3.47
Trying 4.2.2.3.48
Trying 4.2.2.1.19A
Trying RI-GROUP.08.10.02.0017
2 is not created
Trying RWA.014
Trying RWA.007
Trying RWA.015
Trying RWA.016
DataFrame is written to Excel File successfully.


In [7]:
file_names = ["RI-GROUP.08.10.02.0017"]
ws_nr = [ 2]
sqls = [
"""
select 
CASE
            WHEN ENTITY_CD ='HO' THEN 'RBI'
                        ELSE  ENTITY_CD
        END AS ENTITY_CD
        , DQI_CHECK
        ,CAST(ID AS VARCHAR(101)) AS ID
        ,EXPOSURE_AMT_EUR as CUMULATIVE_EXP_EUR
        ,EXP_AMT_ORIG as CUMULATIVE_EXP_ORIG
        ,COALESCE(EXP_CURRENCY,'N/A') AS EXP_CURRENCY
        ,EXP_ATTRIBUTE_5
        ,TOTAL_WCV_AMT_EUR as CUMULATIVE_WCV_EUR
,CASE
            WHEN EXPOSURE_AMT_EUR IS NULL OR TOTAL_WCV_AMT_EUR IS NULL THEN 'NOT OK'
            WHEN TOTAL_WCV_AMT_EUR <= EXPOSURE_AMT_EUR*1.015 THEN 'OK'
            ELSE 'NOT OK'
        END AS ERGEBNIS_0017
from
(
select 
CAST(LDE.CONTRACT_REFERENCE AS VARCHAR(101))             AS ID
,SUBSTR(LDE.BOOK_CODE, 0, INSTR(LDE.BOOK_CODE,'_')-1) AS ENTITY_CD
,CAST('LOANDEPO' AS VARCHAR(50))        AS DQI_CHECK
,CAST(lde.attribute_5  AS VARCHAR(100)) AS EXP_ATTRIBUTE_5
,lde.nominal AS EXP_AMT_ORIG
,CAST(LDE.CURRENCY   AS VARCHAR(3))   AS EXP_CURRENCY
,(LDE.NOMINAL * FX_LDE.EXCH_RATE)         AS EXPOSURE_AMT_EUR
,sum(CGU.AMOUNT_1 * FX_CGU.EXCH_RATE) AS TOTAL_WCV_AMT_EUR
from loandepo lde
JOIN  CURRENCY_EXCH_RATE FX_LDE
            ON FX_LDE.CCY_CODE = LDE.CURRENCY
            AND FX_LDE.CCY_SET_CODE LIKE'EZB_%'
and FX_LDE.PARTITION_KEY='"""+rd+"""0000'
join  contract_guarantee cgu
on CGU.BENEF_CONTRACT_REF = LDE.CONTRACT_REFERENCE
AND    CGU.BENEF_TABLE_NAME = 'LOANDEPO'
and CGU.PARTITION_KEY='"""+rd+"""0000'
JOIN  CURRENCY_EXCH_RATE FX_CGU
            ON FX_CGU.CCY_CODE = CGU.CURRENCY
            AND FX_CGU.CCY_SET_CODE LIKE'EZB_%'
and FX_CGU.PARTITION_KEY='"""+rd+"""0000'
AND (CGU.AMOUNT_1 * FX_CGU.EXCH_RATE) <>0
where (1=1)
and LDE.PARTITION_KEY='"""+rd+"""0000'
group by 
LDE.CONTRACT_REFERENCE,
LDE.BOOK_CODE,
lde.attribute_5,
lde.nominal,
LDE.CURRENCY,
(LDE.NOMINAL * FX_LDE.EXCH_RATE) 
UNION
select 
CAST(FAC.CONTRACT_REFERENCE AS VARCHAR(101))             AS ID
,SUBSTR(FAC.BOOK_CODE, 0, INSTR(FAC.BOOK_CODE,'_')-1) AS ENTITY_CD
 ,CAST('FACILITY' AS VARCHAR(50))        AS DQI_CHECK
,CAST(FAC.attribute_5  AS VARCHAR(100)) AS EXP_ATTRIBUTE_5
,FAC.AMOUNT AS EXP_AMT_ORIG
,CAST(FAC.CURRENCY   AS VARCHAR(3))   AS EXP_CURRENCY
,(FAC.AMOUNT * FX_FAC.EXCH_RATE)         AS EXPOSURE_AMT_EUR
,sum(CGU.AMOUNT_1 * FX_CGU.EXCH_RATE) AS TOTAL_WCV_AMT_EUR
from FACILITY FAC
JOIN  CURRENCY_EXCH_RATE FX_FAC
            ON FX_FAC.CCY_CODE = FAC.CURRENCY
            AND FX_FAC.CCY_SET_CODE LIKE'EZB_%'
and FX_FAC.PARTITION_KEY='"""+rd+"""0000'
join  contract_guarantee cgu
on CGU.BENEF_CONTRACT_REF = FAC.CONTRACT_REFERENCE
AND    CGU.BENEF_TABLE_NAME = 'FACILITY'
and CGU.PARTITION_KEY='"""+rd+"""0000'
JOIN  CURRENCY_EXCH_RATE FX_CGU
            ON FX_CGU.CCY_CODE = CGU.CURRENCY
            AND FX_CGU.CCY_SET_CODE LIKE'EZB_%'
and FX_CGU.PARTITION_KEY='"""+rd+"""0000'
AND (CGU.AMOUNT_1 * FX_CGU.EXCH_RATE) <>0
where (1=1)
and FAC.PARTITION_KEY='"""+rd+"""0000'
group by 
FAC.CONTRACT_REFERENCE,
FAC.BOOK_CODE,
FAC.attribute_5,
FAC.AMOUNT,
FAC.CURRENCY,
(FAC.AMOUNT * FX_FAC.EXCH_RATE) 
UNION
SELECT
CAST(ACC.ACCOUNT_REFERENCE AS VARCHAR(101)) AS ID
,SUBSTR(ACC.BOOK_CODE, 0, INSTR(ACC.BOOK_CODE,'_')-1) AS ENTITY_CD
,CAST('ACCOUNT' AS VARCHAR(50))         AS DQI_CHECK
,CAST(ACC.ATTRIBUTE_5 AS VARCHAR(100)) AS EXP_ATTRIBUTE_5
      ,(COALESCE(ACC.ACCRUALS,0)+ACC.BALANCE) AS EXP_AMT_ORIG
   ,CAST(ACC.CURRENCY AS VARCHAR(3))AS EXP_CURRENCY
   ,(COALESCE(ACC.ACCRUALS,0)+ACC.BALANCE) * FX_ACC.EXCH_RATE AS EXPOSURE_AMT_EUR
	,sum(CGU.AMOUNT_1 * FX_CGU.EXCH_RATE) AS TOTAL_WCV_AMT_EUR
   FROM ACCOUNT ACC
JOIN  CURRENCY_EXCH_RATE FX_ACC
            ON FX_ACC.CCY_CODE = ACC.CURRENCY
            AND FX_ACC.CCY_SET_CODE LIKE'EZB_%'
and FX_ACC.PARTITION_KEY='"""+rd+"""0000'
JOIN contract_guarantee CGU
ON ACC.ACCOUNT_REFERENCE = CGU.BENEF_CONTRACT_REF
AND    CGU.BENEF_TABLE_NAME = 'ACCOUNT'
and CGU.PARTITION_KEY='"""+rd+"""0000'
JOIN  CURRENCY_EXCH_RATE FX_CGU
            ON FX_CGU.CCY_CODE = CGU.CURRENCY
            AND FX_CGU.CCY_SET_CODE LIKE'EZB_%'
and FX_CGU.PARTITION_KEY='"""+rd+"""00000'
AND (CGU.AMOUNT_1 * FX_CGU.EXCH_RATE) <>0
where (1=1)
and ACC.PARTITION_KEY='"""+rd+"""0000'
group by 
ACC.ACCOUNT_REFERENCE ,
ACC.BOOK_CODE,
ACC.attribute_5,
(COALESCE(ACC.ACCRUALS,0)+ACC.BALANCE),
ACC.CURRENCY,
(COALESCE(ACC.ACCRUALS,0)+ACC.BALANCE) * FX_ACC.EXCH_RATE
UNION
select 
CAST(OOB.CONTRACT_REFERENCE AS VARCHAR(101))             AS ID
,SUBSTR(OOB.BOOK_CODE, 0, INSTR(OOB.BOOK_CODE,'_')-1) AS ENTITY_CD
 ,CAST('ON_OFF_BALANCE' AS VARCHAR(50))        AS DQI_CHECK
,CAST(OOB.attribute_5  AS VARCHAR(100)) AS EXP_ATTRIBUTE_5
,OOB.OUTSTANDING AS EXP_AMT_ORIG
,CAST(OOB.CURRENCY   AS VARCHAR(3))   AS EXP_CURRENCY
,(OOB.OUTSTANDING* FX_OOB.EXCH_RATE)         AS EXPOSURE_AMT_EUR
,sum(CGU.AMOUNT_1 * FX_CGU.EXCH_RATE) AS TOTAL_WCV_AMT_EUR
from ON_OFF_BALANCE OOB
JOIN  CURRENCY_EXCH_RATE FX_OOB
            ON FX_OOB.CCY_CODE = OOB.CURRENCY
            AND FX_OOB.CCY_SET_CODE LIKE'EZB_%'
and FX_OOB.PARTITION_KEY='"""+rd+"""0000'
join  contract_guarantee cgu
on CGU.BENEF_CONTRACT_REF = OOB.CONTRACT_REFERENCE
AND    CGU.BENEF_TABLE_NAME = 'ON_OFF_BALANCE'
and CGU.PARTITION_KEY='"""+rd+"""0000'
JOIN  CURRENCY_EXCH_RATE FX_CGU
            ON FX_CGU.CCY_CODE = CGU.CURRENCY
            AND FX_CGU.CCY_SET_CODE LIKE'EZB_%'
and FX_CGU.PARTITION_KEY='"""+rd+"""0000'
AND (CGU.AMOUNT_1 * FX_CGU.EXCH_RATE) <>0
where (1=1)
and OOB.PARTITION_KEY='"""+rd+"""0000'
group by 
OOB.CONTRACT_REFERENCE,
OOB.BOOK_CODE,
OOB.attribute_5,
OOB.OUTSTANDING,
OOB.CURRENCY,
(OOB.OUTSTANDING* FX_OOB.EXCH_RATE) 

UNION
select 
CAST(FX.CONTRACT_REFERENCE AS VARCHAR(101))             AS ID
,SUBSTR(FX.BOOK_CODE, 0, INSTR(FX.BOOK_CODE,'_')-1) AS ENTITY_CD
 ,CAST('FOREX' AS VARCHAR(50))        AS DQI_CHECK
,CAST(FX.attribute_5  AS VARCHAR(100)) AS EXP_ATTRIBUTE_5
,FX.PURCHASE_CCY_NOMINAL AS EXP_AMT_ORIG
,CAST(FX.PURCHASE_CCY   AS VARCHAR(3))   AS EXP_CURRENCY
,(FX.PURCHASE_CCY_NOMINAL* FX_FX.EXCH_RATE)         AS EXPOSURE_AMT_EUR
,sum(CGU.AMOUNT_1 * FX_CGU.EXCH_RATE) AS TOTAL_WCV_AMT_EUR
from FOREX FX
JOIN  CURRENCY_EXCH_RATE FX_FX
            ON FX_FX.CCY_CODE = FX.PURCHASE_CCY
            AND FX_FX.CCY_SET_CODE LIKE'EZB_%'
and FX_FX.PARTITION_KEY='"""+rd+"""0000'
join  contract_guarantee cgu
on CGU.BENEF_CONTRACT_REF = FX.CONTRACT_REFERENCE
AND    CGU.BENEF_TABLE_NAME = 'FOREX'
and CGU.PARTITION_KEY='"""+rd+"""0000'
JOIN  CURRENCY_EXCH_RATE FX_CGU
            ON FX_CGU.CCY_CODE = FX.PURCHASE_CCY
            AND FX_CGU.CCY_SET_CODE LIKE'EZB_%'
and FX_CGU.PARTITION_KEY='"""+rd+"""0000'
AND (CGU.AMOUNT_1 * FX_CGU.EXCH_RATE) <>0
where (1=1)
and FX.PARTITION_KEY='"""+rd+"""0000'
group by 
FX.CONTRACT_REFERENCE,
FX.BOOK_CODE,
FX.attribute_5,
FX.PURCHASE_CCY_NOMINAL,
FX.PURCHASE_CCY,
(FX.PURCHASE_CCY_NOMINAL* FX_FX.EXCH_RATE) 

UNION
select 
CAST(REC.CONTRACT_REFERENCE AS VARCHAR(101))             AS ID
,SUBSTR(REC.BOOK_CODE, 0, INSTR(REC.BOOK_CODE,'_')-1) AS ENTITY_CD
 ,CAST('RECEIVABLE' AS VARCHAR(50))        AS DQI_CHECK
,CAST(REC.attribute_5  AS VARCHAR(100)) AS EXP_ATTRIBUTE_5
,REC.nominal AS EXP_AMT_ORIG
,CAST(REC.CURRENCY   AS VARCHAR(3))   AS EXP_CURRENCY
,(REC.NOMINAL * FX_REC.EXCH_RATE)         AS EXPOSURE_AMT_EUR
,sum(CGU.AMOUNT_1 * FX_CGU.EXCH_RATE) AS TOTAL_WCV_AMT_EUR
from RECEIVABLE REC
JOIN  CURRENCY_EXCH_RATE FX_REC
            ON FX_REC.CCY_CODE = REC.CURRENCY
            AND FX_REC.CCY_SET_CODE LIKE'EZB_%'
and FX_REC.PARTITION_KEY='"""+rd+"""0000'
join  contract_guarantee cgu
on CGU.BENEF_CONTRACT_REF = REC.CONTRACT_REFERENCE
AND    CGU.BENEF_TABLE_NAME = 'RECEIVABLE'
and CGU.PARTITION_KEY='"""+rd+"""0000'
JOIN  CURRENCY_EXCH_RATE FX_CGU
            ON FX_CGU.CCY_CODE = CGU.CURRENCY
            AND FX_CGU.CCY_SET_CODE LIKE'EZB_%'
and FX_CGU.PARTITION_KEY='"""+rd+"""0000'
AND (CGU.AMOUNT_1 * FX_CGU.EXCH_RATE) <>0
where (1=1)
and REC.PARTITION_KEY='"""+rd+"""0000'
group by 
REC.CONTRACT_REFERENCE,
REC.BOOK_CODE,
REC.attribute_5,
REC.nominal,
REC.CURRENCY,
(REC.NOMINAL * FX_REC.EXCH_RATE) 
UNION
select 
CAST(DOC.CONTRACT_REFERENCE AS VARCHAR(101))             AS ID
,SUBSTR(DOC.BOOK_CODE, 0, INSTR(DOC.BOOK_CODE,'_')-1) AS ENTITY_CD
 ,CAST('DOC_CREDIT' AS VARCHAR(50))        AS DQI_CHECK
,CAST(DOC.attribute_5  AS VARCHAR(100)) AS EXP_ATTRIBUTE_5
,DOC.CONTRACT_AMOUNT AS EXP_AMT_ORIG
,CAST(DOC.CURRENCY_CODE   AS VARCHAR(3))   AS EXP_CURRENCY
,(DOC.CONTRACT_AMOUNT * FX_DOC.EXCH_RATE)         AS EXPOSURE_AMT_EUR
,sum(CGU.AMOUNT_1 * FX_CGU.EXCH_RATE) AS TOTAL_WCV_AMT_EUR
from DOC_CREDIT DOC
JOIN  CURRENCY_EXCH_RATE FX_DOC
            ON FX_DOC.CCY_CODE = DOC.CURRENCY_CODE
            AND FX_DOC.CCY_SET_CODE LIKE'EZB_%'
and FX_DOC.PARTITION_KEY='"""+rd+"""0000'
join  contract_guarantee cgu
on CGU.BENEF_CONTRACT_REF = DOC.CONTRACT_REFERENCE
AND    CGU.BENEF_TABLE_NAME = 'DOC_CREDIT'
and CGU.PARTITION_KEY='"""+rd+"""0000'
JOIN  CURRENCY_EXCH_RATE FX_CGU
            ON FX_CGU.CCY_CODE = CGU.CURRENCY
            AND FX_CGU.CCY_SET_CODE LIKE'EZB_%'
and FX_CGU.PARTITION_KEY='"""+rd+"""0000'
AND (CGU.AMOUNT_1 * FX_CGU.EXCH_RATE) <>0
where (1=1)
and DOC.PARTITION_KEY='"""+rd+"""0000'
group by 
DOC.CONTRACT_REFERENCE,
DOC.BOOK_CODE,
DOC.attribute_5,
DOC.CONTRACT_AMOUNT,
DOC.CURRENCY_CODE,
(DOC.CONTRACT_AMOUNT * FX_DOC.EXCH_RATE) 
)
where (1=1)
AND TOTAL_WCV_AMT_EUR <> 0.00
"""
]
print(str(sqls[0]))
#save_file(file_names, ws_nr, sqls)


select 
CASE
            WHEN ENTITY_CD ='HO' THEN 'RBI'
                        ELSE  ENTITY_CD
        END AS ENTITY_CD
        , DQI_CHECK
        ,CAST(ID AS VARCHAR(101)) AS ID
        ,EXPOSURE_AMT_EUR as CUMULATIVE_EXP_EUR
        ,EXP_AMT_ORIG as CUMULATIVE_EXP_ORIG
        ,COALESCE(EXP_CURRENCY,'N/A') AS EXP_CURRENCY
        ,EXP_ATTRIBUTE_5
        ,TOTAL_WCV_AMT_EUR as CUMULATIVE_WCV_EUR
,CASE
            WHEN EXPOSURE_AMT_EUR IS NULL OR TOTAL_WCV_AMT_EUR IS NULL THEN 'NOT OK'
            WHEN TOTAL_WCV_AMT_EUR <= EXPOSURE_AMT_EUR*1.015 THEN 'OK'
            ELSE 'NOT OK'
        END AS ERGEBNIS_0017
from
(
select 
CAST(LDE.CONTRACT_REFERENCE AS VARCHAR(101))             AS ID
,SUBSTR(LDE.BOOK_CODE, 0, INSTR(LDE.BOOK_CODE,'_')-1) AS ENTITY_CD
,CAST('LOANDEPO' AS VARCHAR(50))        AS DQI_CHECK
,CAST(lde.attribute_5  AS VARCHAR(100)) AS EXP_ATTRIBUTE_5
,lde.nominal AS EXP_AMT_ORIG
,CAST(LDE.CURRENCY   AS VARCHAR(3))   AS EXP_CURRENCY
,(LDE.NOMINAL * FX_LDE.EXCH_RATE)      

### Zipping files

In [16]:
# create a ZipFile object
zip_name = 'DQIs for ' + rd[:6] + '.zip'
zipObj = ZipFile(ex_folder + zip_name, 'w')

for file_name in file_names:
    zipObj.write(ex_folder + "Export_" + file_name + "_" + rd[:6] + ".csv")

# close the Zip File
zipObj.close()

### Emailing DQI

In [10]:
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = 'patrick.zadnik@rbinternational.com'
mail.Cc = 'markus.hochleitner@rbinternational.com; martina.djurovic@rbinternational.com'
mail.Subject = 'DQIs based on OCR'
mail.Body = '''Dear Patrick,

Please find attached the Retail data as of ''' + rd + '''.

Thanks in advance and kind regards,
Arber Bajraktari
Student Support IT

Raiffeisen Bank International AG | Am Stadtpark 9, 1030 Vienna, Austria | Tel. : +43 664 99481391
arber.bajraktari@rbinternational.com | http://www.rbinternational.com 
'''
# To attach a file to the email (optional):
#attachment  = directory + "/" + zip_name
#mail.Attachments.Add(attachment)
mail.Display()